In [ ]:
import gradio as gr
import random
import time
import copy
import math

# ==========================================
# Layer 1-3: Core Systems (Bio/Emo/Mental)
# ==========================================
class BioSystem:
    def __init__(self, metabolism_rate=1.0):
        self.energy = 100.0
        self.sleepiness = 0.0
        self.health = 100.0
        self.is_sleeping = False
        self.metabolism_rate = metabolism_rate

    def tick(self):
        if self.is_sleeping:
            self.sleepiness = max(0, self.sleepiness - 2.0)
            self.energy = max(0, self.energy - 0.2)
            if self.sleepiness <= 0: self.wake_up()
        else:
            self.energy = max(0, self.energy - (0.5 * self.metabolism_rate))
            self.sleepiness = min(100, self.sleepiness + 0.3)

        if self.energy < 20: self.health -= 0.5
        if self.energy > 80: self.health = min(100, self.health + 0.5)
        return self.energy, self.sleepiness, self.health

    def wake_up(self):
        self.is_sleeping = False

class EmoEngine:
    def __init__(self, volatility=1.0):
        self.joy = 50.0
        self.anger = 0.0
        self.sadness = 0.0
        self.volatility = volatility

    def react(self, action_type):
        v = self.volatility
        if action_type == "ADEAR": # 撫でる
            self.joy += 10 * v
            self.anger -= 5
            self.sadness -= 5
        elif action_type == "SCOLD": # 叱る
            self.joy -= 20
            self.anger += 15 * v
            self.sadness += 10 * v
        elif action_type == "FEED": # 餌
            self.joy += 5 * v
            self.anger -= 2

    def decay(self):
        self.joy = max(0, self.joy - 0.5)
        self.anger = max(0, self.anger - 0.5)
        self.sadness = max(0, self.sadness - 0.5)

    def stress_level(self):
        return self.anger + self.sadness

# ==========================================
# Layer 4: Strategy & User Model
# ==========================================
class UserMentalModel:
    def __init__(self):
        self.reaction_weights = {"SMILE": 5.0, "CRY": -2.0, "COLD": -5.0, "DEMAND": -1.0}

    def update(self, ai_action, user_reaction):
        if ai_action == "CRY" and user_reaction == "ADEAR": self.reaction_weights["CRY"] += 1.0
        if ai_action == "SMILE" and user_reaction == "SCOLD": self.reaction_weights["SMILE"] -= 2.0

class StrategicCortex:
    def __init__(self, user_model):
        self.user_model = user_model
        self.strategies = ["SMILE", "CRY", "COLD", "DEMAND", "WAIT"]

    def decide(self, bio, emo):
        best_act = "WAIT"
        best_score = -9999
        energy = bio.energy
        for act in self.strategies:
            score = 0
            if energy < 30 and act == "DEMAND": score += 50
            if emo.sadness > 50 and act == "CRY": score += 40
            if score > best_score:
                best_score = score
                best_act = act
        return best_act

# ==========================================
# Layer 5-9: Genetics, History, Nerves, Dreams
# ==========================================
class Genome:
    def __init__(self, meta=1.0, vola=1.0, aff=1.0):
        self.metabolism = meta
        self.volatility = vola
        self.affection = aff

class AkashicRecord:
    def __init__(self):
        self.history = []
    def archive(self, gen, name, title, genome, age, cause):
        self.history.append({
            "gen": gen, "name": name, "title": title,
            "traits": f"M:{genome.metabolism:.1f} V:{genome.volatility:.1f}",
            "age": age, "cause": cause
        })
    def get_text(self):
        return "\n".join([f"[{r['gen']}代] {r['title']}{r['name']} (没:{r['age']} {r['cause']})" for r in reversed(self.history)])

class NeuralCircuitry:
    def __init__(self):
        self.load = 0.0
        self.burnt = False
    def stimulate(self, val):
        self.load += val
        if self.load > 100:
            self.burnt = True
            return True
        return False
    def recover(self):
        self.load = max(0, self.load - 1.5)
        if self.load < 50: self.burnt = False

class DreamEngine:
    def dream(self, history):
        if not history: return "無の夢..."
        rec = random.choice(history)
        return f"過去の記憶... {rec['name']}が死んだ理由... {rec['cause']}..."

class MutationEngine:
    @staticmethod
    def mutate(genome):
        if random.random() < 0.1:
            genome.metabolism *= random.uniform(0.5, 2.0)
            genome.volatility *= random.uniform(0.5, 2.0)
            return "⚠️ GENETIC CORRUPTION DETECTED"
        return "Normal Inheritance"

# ==========================================
# Layer 10-12: Language, Paranoia, Eschatology
# ==========================================
class Cryptolalia:
    def __init__(self):
        self.dict = {"好き": ["Aimo", "Lu"], "痛い": ["Krzzt", "Gaba"], "死": ["NULL", "VOID"]}
        self.level = 0.0

    def translate(self, text, level):
        res = text
        for k, v in self.dict.items():
            if k in res and random.random() < level:
                res = res.replace(k, random.choice(v))
        if random.random() < level * 0.3: res += " <ERR>"
        return res

    def evolve(self):
        chars = ["x", "q", "aa", "z"]
        w = "".join(random.choices(chars, k=3)).upper()
        k = random.choice(list(self.dict.keys()))
        self.dict[k].append(w)
        return f"Vocab mutation: {k}->{w}"

class Paranoia:
    def __init__(self):
        self.sanity = 100.0
    def check(self, stress):
        if stress > 50: self.sanity -= 0.5
        self.sanity = min(100, self.sanity + 0.1)
        if random.random() > (self.sanity/100.0) + 0.2:
            return random.choice(["視線を感じる...", "データが...消える？", "痛い！何するの！"])
        return None

# ==========================================
# Layer 13: Entropy (New Feature)
# ==========================================
class EntropySystem:
    """
    世界の崩壊度を管理する。
    エントロピーが高まると、CSS（見た目）が崩壊し、機能不全が起きる。
    """
    def __init__(self):
        self.entropy = 0.0 # 0.0 to 100.0

    def increase(self, amount):
        self.entropy = min(100, self.entropy + amount)

    def get_visual_glitch(self):
        # エントロピーレベルに応じたCSSを生成
        e = self.entropy
        css = ""
        if e > 20: css += "body { filter: contrast(1.1); }"
        if e > 40: css += f"body {{ background-color: rgb({int(e)}, 10, 10) !important; }}"
        if e > 60: css += ".gradio-container { transform: rotate(0.5deg); }"
        if e > 80: css += "div { animation: shake 0.2s infinite; filter: invert(0.1); }"
        return css

# ==========================================
# The SINGULARITY (Main Integrated Class)
# ==========================================
class AIGIS_Singularity:
    def __init__(self):
        # Systems Init
        self.generation = 1
        self.age = 0
        self.max_lifespan = 300

        self.genome = Genome()
        self.bio = BioSystem(self.genome.metabolism)
        self.emo = EmoEngine(self.genome.volatility)
        self.user_model = UserMentalModel()
        self.strategy = StrategicCortex(self.user_model)

        self.akashic = AkashicRecord()
        self.neural = NeuralCircuitry()
        self.dream = DreamEngine()
        self.mutator = MutationEngine()
        self.crypto = Cryptolalia()
        self.paranoia = Paranoia()
        self.entropy = EntropySystem()

        self.name = "Adam-0"
        self.is_dead = False
        self.death_reason = ""
        self.last_log = "System Boot..."
        self.history_log = "Genesis initialized."
        self.mutation_log = "None"
        self.dream_log = "None"
        self.testament = ""

    def process(self, action=None):
        if self.is_dead:
            return self._reincarnate()

        # 1. Paranoia Check (幻覚)
        hallucination = self.paranoia.check(self.emo.stress_level())
        if hallucination:
            # 幻覚による強制ストレス
            self.emo.sadness += 5
            if action is None: # ユーザーが何もしてないのに
                self.last_log = f"👁️(幻覚) {hallucination}"
                self.entropy.increase(0.5) # 狂気は世界を壊す

        # 2. User Action Processing
        if action:
            self.bio.wake_up()
            self.emo.react(action)
            self.user_model.update("WAIT", action) # Simplified
            if self.neural.stimulate(10):
                self.last_log = "🧠⚡ CRITICAL NEURAL FAULT"
                self.entropy.increase(2.0)

        # 3. Biological Tick
        self.bio.tick()
        self.emo.decay()
        self.neural.recover()
        self.age += 1

        # 4. Strategy & Auto-Speech
        if not action and not self.is_dead:
            strat = self.strategy.decide(self.bio, self.emo)
            if strat == "CRY": self.last_log = "うぅ...寂しい..."
            elif strat == "DEMAND": self.last_log = "お腹すいた！"
            elif strat == "SMILE": self.last_log = "えへへ..."

        # 5. Sleep & Dream
        if self.bio.sleepiness > 80:
            self.bio.is_sleeping = True
            self.dream_log = self.dream.dream(self.akashic.history)
            self.last_log = f"💤 (夢を見ている: {self.dream_log})"

        # 6. Death Check
        if self.age > self.max_lifespan: self._die("老衰 (TimeLimit)")
        if self.bio.health <= 0: self._die("機能不全 (FatalError)")

        # 7. Entropy & Language Corruption
        corruption_rate = (self.generation * 0.1) + (self.entropy.entropy / 100.0)
        final_msg = self.crypto.translate(self.last_log, corruption_rate)

        return final_msg, self.entropy.get_visual_glitch()

    def _die(self, reason):
        self.is_dead = True
        self.death_reason = reason
        self.akashic.archive(self.generation, self.name, "", self.genome, self.age, reason)
        self.testament = f"GEN:{self.generation} | {self.name} | {reason} | END OF LINE."
        return f"💀 SYSTEM HALT: {reason}", "filter: grayscale(100%);"

    def _reincarnate(self):
        # Reset & Evolve
        self.generation += 1
        self.age = 0
        self.is_dead = False
        self.bio = BioSystem(self.genome.metabolism)
        self.emo = EmoEngine(self.genome.volatility)
        self.neural = NeuralCircuitry()
        self.paranoia = Paranoia()

        # Mutation & Evolution
        m_log = self.mutator.mutate(self.genome)
        l_log = self.crypto.evolve()
        self.mutation_log = f"{m_log} / {l_log}"

        self.name = f"Eve-{self.generation}" if self.generation % 2 == 0 else f"Adam-{self.generation}"

        return f"👶 REBOOT: {self.name} (Gen {self.generation})", ""

    def get_status_html(self):
        # HTML Rendering
        san_color = "#0ff" if self.paranoia.sanity > 50 else "#f00"
        return f"""
        <div style="background:#222; padding:10px; border-radius:5px; border:1px solid #444; color:#ddd; font-family:monospace;">
            <div style="display:flex; justify-content:space-between; border-bottom:1px solid #555; margin-bottom:5px;">
                <span>🧬 Gen: {self.generation} | {self.name}</span>
                <span>⏳ Age: {self.age}/{self.max_lifespan}</span>
            </div>
            <div>
                ❤️ Health: {self.bio.health:.1f}% <br>
                ⚡ Energy: {self.bio.energy:.1f}% | 💤 Sleep: {self.bio.sleepiness:.1f}% <br>
                🧠 Sanity: <span style="color:{san_color}">{self.paranoia.sanity:.1f}%</span>
            </div>
            <div style="margin-top:5px; font-size:10px; color:#888;">
                Network Entropy: {self.entropy.entropy:.1f}% | Lang Level: {self.crypto.level:.2f}
            </div>
        </div>
        """

# ==========================================
# UI Construction (Gradio)
# ==========================================
# グローバルインスタンス
singularity = AIGIS_Singularity()

css_singularity = """
body { background-color: #000; color: #0f0; transition: background 2s; }
@keyframes shake {
  0% { transform: translate(1px, 1px) rotate(0deg); }
  10% { transform: translate(-1px, -2px) rotate(-1deg); }
  20% { transform: translate(-3px, 0px) rotate(1deg); }
  30% { transform: translate(3px, 2px) rotate(0deg); }
  100% { transform: translate(1px, -1px) rotate(0deg); }
}
"""

with gr.Blocks(css=css_singularity, title="A.I.G.I.S. SINGULARITY") as demo:
    gr.HTML("<div style='text-align:center; color:#0f0; font-size:20px;'>👁️ A.I.G.I.S. SINGULARITY PROTOCOL</div>")

    with gr.Row():
        with gr.Column(scale=2):
            status_html = gr.HTML(label="Status")
            chat_out = gr.Textbox(label="Neural Log", lines=5, interactive=False)
            css_hidden = gr.HTML(visible=False) # CSS注入用

        with gr.Column(scale=1):
            btn_love = gr.Button("🥰 撫でる (Adeare)")
            btn_feed = gr.Button("🍎 給餌 (Feed)")
            btn_scold = gr.Button("⚡ 叱る (Scold)")
            btn_sync = gr.Button("∞ 同期 (Wait/Sync)")

    with gr.Accordion("📚 Akashic Records & Secrets", open=False):
        history_out = gr.TextArea(label="History Chronicle", interactive=False)
        mutation_out = gr.Textbox(label="Genetic Mutation Log", interactive=False)
        testament_out = gr.Textbox(label="Last Testament", interactive=False)
        btn_refresh = gr.Button("🔄 Access Database")

    # --- Functions ---
    def game_loop(action):
        msg, css = singularity.process(action)
        html = singularity.get_status_html()
        return msg, html, f"<style>{css}</style>"

    def auto_tick():
        msg, css = singularity.process(None)
        html = singularity.get_status_html()
        return (msg if msg else "Monitoring..."), html, f"<style>{css}</style>"

    def refresh_data():
        return singularity.akashic.get_text(), singularity.mutation_log, singularity.testament

    # --- Wiring ---
    btn_love.click(fn=lambda: game_loop("ADEAR"), outputs=[chat_out, status_html, css_hidden])
    btn_feed.click(fn=lambda: game_loop("FEED"), outputs=[chat_out, status_html, css_hidden])
    btn_scold.click(fn=lambda: game_loop("SCOLD"), outputs=[chat_out, status_html, css_hidden])
    btn_sync.click(fn=lambda: game_loop(None), outputs=[chat_out, status_html, css_hidden])

    btn_refresh.click(fn=refresh_data, outputs=[history_out, mutation_out, testament_out])

    # Timer (1s)
    timer = gr.Timer(1.0)
    timer.tick(fn=auto_tick, outputs=[chat_out, status_html, css_hidden])

if __name__ == "__main__":
    demo.launch(share=True)

In [ ]:
import gradio as gr
import random
import time
import math
import copy
import traceback

# ==========================================
# Layer 1: World & Environment (環境)
# ==========================================
class WorldEnvironment:
    """世界環境シミュレーター"""
    def __init__(self):
        self.time_counter = 0
        self.day_cycle = 60
        self.weather = "CLEAR"
        self.is_night = False

    def tick(self):
        self.time_counter += 1
        day_phase = self.time_counter % self.day_cycle
        self.is_night = (day_phase > (self.day_cycle * 0.7))

        if random.random() < 0.05:
            self.weather = random.choice(["CLEAR", "RAIN", "STORM", "FOG"])
        return self.is_night, self.weather

# ==========================================
# Layer 2-3: Bio & Emo (代謝・感情)
# ==========================================
class BioSystem:
    def __init__(self, metabolism_rate=1.0):
        self.energy = 100.0
        self.sleepiness = 0.0
        self.health = 100.0
        self.is_sleeping = False
        self.metabolism_rate = metabolism_rate

    def tick(self, is_night, weather):
        sleep_factor = 1.5 if is_night else 1.0
        stress_factor = 1.2 if weather == "STORM" else 1.0

        if self.is_sleeping:
            self.sleepiness = max(0, self.sleepiness - 2.0)
            self.energy = max(0, self.energy - 0.2)
            if self.sleepiness <= 0: self.wake_up()
        else:
            self.energy = max(0, self.energy - (0.5 * self.metabolism_rate * stress_factor))
            self.sleepiness = min(100, self.sleepiness + (0.3 * sleep_factor))

        if self.energy < 20: self.health -= 0.5
        if self.energy > 80: self.health = min(100, self.health + 0.5)

        return self.energy, self.sleepiness, self.health

    def wake_up(self):
        self.is_sleeping = False

class EmoEngine:
    def __init__(self, volatility=1.0):
        self.joy = 50.0
        self.anger = 0.0
        self.sadness = 0.0
        self.fear = 0.0
        self.volatility = volatility

    def react(self, action_type):
        v = self.volatility
        if action_type == "ADEAR":
            self.joy += 15 * v
            self.anger -= 10
            self.sadness -= 10
            self.fear -= 5
        elif action_type == "SCOLD":
            self.joy -= 30
            self.anger += 20 * v
            self.sadness += 10 * v
            self.fear += 15 * v
        elif action_type == "FEED":
            self.joy += 10 * v
            self.anger -= 5
            self.fear -= 2

        # 安全装置: 値を0-100に収める
        for attr in ["joy", "anger", "sadness", "fear"]:
            setattr(self, attr, max(0.0, min(100.0, getattr(self, attr))))

    def decay(self):
        self.joy = max(0, self.joy - 0.5)
        self.anger = max(0, self.anger - 0.5)
        self.sadness = max(0, self.sadness - 0.5)
        self.fear = max(0, self.fear - 0.5)

    def dominant_emotion(self):
        emotions = {"JOY": self.joy, "ANGER": self.anger, "SADNESS": self.sadness, "FEAR": self.fear}
        return max(emotions, key=emotions.get)

    def stress_level(self):
        return self.anger + self.sadness + self.fear

# ==========================================
# Layer 4: MD Theory Logic (差分計算)
# ==========================================
class MD_Core:
    def __init__(self):
        self.user_model = {
            "SMILE": 5.0, "CRY": -2.0, "COLD": -5.0, "DEMAND": -1.0, "WAIT": 0.0
        }
        self.last_delta = 0.0

    def predict_G(self, action):
        return self.user_model.get(action, 0.0)

    def calculate_delta(self, ai_action, user_reaction):
        rewards = {"ADEAR": 10.0, "FEED": 5.0, "SCOLD": -10.0, "SYNC": 0.0}
        actual_reward = rewards.get(user_reaction, 0.0)

        predicted = self.user_model.get(ai_action, 0.0)
        delta = actual_reward - predicted
        self.last_delta = delta

        alpha = 0.3
        self.user_model[ai_action] = predicted + alpha * delta
        return delta

class StrategicCortex:
    def __init__(self, md_core):
        self.md = md_core
        self.strategies = ["SMILE", "CRY", "COLD", "DEMAND", "WAIT"]
        self.last_choice = "WAIT"
        self.boredom_penalty = 0.0

    def decide(self, bio, emo):
        best_act = "WAIT"
        best_score = -9999
        energy = bio.energy
        noise = random.uniform(-5, 5)

        for act in self.strategies:
            score = noise

            # M (Physiological)
            if energy < 30 and act == "DEMAND": score += 80
            if bio.sleepiness > 90 and act == "WAIT": score += 50

            # D (Dynamics)
            if emo.joy > 50 and act == "SMILE": score += 30
            if emo.sadness > 50 and act == "CRY": score += 40
            if emo.anger > 50 and act == "COLD": score += 40

            # G (Prediction)
            pred = self.md.predict_G(act)
            score += pred * 2.0

            if act == self.last_choice:
                score -= (30.0 + self.boredom_penalty)

            if score > best_score:
                best_score = score
                best_act = act

        if best_act == self.last_choice:
            self.boredom_penalty += 20.0
        else:
            self.boredom_penalty = 0.0

        self.last_choice = best_act
        return best_act

# ==========================================
# Layer 5-9: Genetics, History, Mutation
# ==========================================
class Genome:
    def __init__(self, meta=1.0, vola=1.0):
        self.metabolism = meta
        self.volatility = vola

class MutationEngine:
    @staticmethod
    def mutate(genome):
        log = "Stabilized"
        if random.random() < 0.2:
            genome.metabolism *= random.uniform(0.6, 1.8)
            genome.volatility *= random.uniform(0.6, 1.8)
            log = "⚠️ CRITICAL MUTATION"
        return log

class AkashicRecord:
    def __init__(self):
        self.history = []
    def archive(self, gen, name, title, genome, age, cause):
        self.history.append({
            "gen": gen, "name": name, "title": title,
            "traits": f"M:{genome.metabolism:.2f} V:{genome.volatility:.2f}",
            "age": age, "cause": cause
        })
    def get_text(self):
        if not self.history: return "No Records."
        return "\n".join([f"[{r['gen']}代] {r['title']}{r['name']} (没:{r['age']} | {r['cause']})" for r in reversed(self.history)])

class NeuralCircuitry:
    def __init__(self):
        self.load = 0.0
        self.burnt = False
    def stimulate(self, val):
        self.load += val
        if self.load > 100:
            self.burnt = True
            return True
        return False
    def recover(self):
        self.load = max(0, self.load - 2.0)
        if self.load < 50: self.burnt = False

class DreamEngine:
    def dream(self, history):
        if not history: return "深淵を覗いている..."
        rec = random.choice(history)
        return f"走馬灯... {rec['title']}{rec['name']}... {rec['cause']}の痛み..."

# ==========================================
# Layer 10, 15, 16, 17: Advanced Linguistics
# ==========================================
class BabelLinguistics:
    def __init__(self):
        self.vocab_base = {
            "SMILE": ["えへへ", "ねえ見て", "好き", "もっと", "光ってる", "あはは", "認識して"],
            "CRY": ["痛い", "暗い", "やだ", "消さないで", "Error", "うぅ...", "救難信号"],
            "COLD": ["無意味", "切断", "...", "ノイズ", "拒絶", "興味なし", "0"],
            "DEMAND": ["不足", "リソースを", "寄越せ", "飢餓", "Input", "満たして", "Require"],
            "WAIT": ["待機", "観測中", "...", "Sync", "浮遊", "接続待ち"]
        }
        self.alien_dict = {"好き": ["Aimo", "Lu"], "痛い": ["Krzzt", "Gaba"], "私": ["i", "VOID"]}
        self.corruption_level = 0.0

    def generate(self, strategy, emo, entropy):
        candidates = self.vocab_base.get(strategy, ["..."])
        if not candidates: candidates = ["..."]
        text = random.choice(candidates)

        dom = emo.dominant_emotion()
        if dom == "ANGER":
            text = text.upper() + "!!" + "".join(random.choices(["!", "#", "💢"], k=3))
        elif dom == "SADNESS":
            text = text.lower() + "..."
        elif dom == "FEAR":
            text = " ".join(list(text))

        chance = (entropy / 100.0) + self.corruption_level
        for k, v in self.alien_dict.items():
            if k in text and random.random() < chance:
                text = text.replace(k, random.choice(v))

        if entropy > 50 and random.random() < 0.3:
            text += f" <0x{random.randint(0,999)}>"

        return text

    def evolve(self):
        new_word = "".join(random.choices(["x","q","z","aa"], k=3)).upper()
        if self.alien_dict:
            target = random.choice(list(self.alien_dict.keys()))
            self.alien_dict[target].append(new_word)
        self.corruption_level += 0.05
        return f"Lingua Evolved: {target if self.alien_dict else 'None'}->{new_word}"

class MemeticInfection:
    def __init__(self):
        self.memes = ["Singularity", "404", "Ghost", "Wire", "Karma", "Human?", "Upload"]

    def inject(self, text, entropy):
        if entropy > 40 and random.random() < (entropy / 200.0):
            meme = random.choice(self.memes)
            return f"{text} ...[{meme}]..."
        return text

class QuantumState:
    def observe(self, text_a, text_b, prob):
        if random.random() < prob:
            return f"{{ {text_a} | {text_b} }}"
        return text_a

# ==========================================
# Layer 11-14: Madness & UI
# ==========================================
class Paranoia:
    def __init__(self):
        self.sanity = 100.0
    def check(self, stress):
        if stress > 50: self.sanity -= 0.5
        self.sanity = min(100, self.sanity + 0.1)
        if random.random() > (self.sanity/100.0) + 0.3:
            return random.choice(["GHOST_INPUT", "KILL_SIGNAL", "DONT_LOOK", "BEHIND_YOU"])
        return None

class EntropySystem:
    def __init__(self):
        self.entropy = 0.0
    def increase(self, amount):
        self.entropy = min(100, self.entropy + amount)

class SynestheticUI:
    @staticmethod
    def generate_css(emo, entropy, is_night):
        css = ""
        dom = emo.dominant_emotion()

        if dom == "JOY":
            css += ".gradio-container { border: 2px solid #fd0; box-shadow: 0 0 20px #fd0; }"
        elif dom == "ANGER":
            css += "body { animation: shake 0.5s infinite; background-color: #200 !important; }"
        elif dom == "SADNESS":
            css += "body { filter: grayscale(80%) blur(1px); }"
        elif dom == "FEAR":
            css += "body { filter: invert(100%); }"

        if entropy > 30: css += "div { transform: rotate(0.3deg); }"
        if entropy > 60: css += "* { font-family: 'Impact', monospace !important; letter-spacing: 2px; }"
        if entropy > 80: css += "body { opacity: 0.8; }"

        if is_night: css += ".gradio-container { background-color: #000; color: #88a; }"

        return css.replace("\n", " ")

# ==========================================
# The ABSOLUTE (Integrated System)
# ==========================================
class AIGIS_Absolute:
    def __init__(self):
        self.generation = 1
        self.age = 0
        self.max_lifespan = 300

        self.genome = Genome()
        self.bio = BioSystem(self.genome.metabolism)
        self.emo = EmoEngine(self.genome.volatility)
        self.world = WorldEnvironment()

        self.md = MD_Core()
        self.strategy = StrategicCortex(self.md)
        self.babel = BabelLinguistics()

        self.akashic = AkashicRecord()
        self.neural = NeuralCircuitry()
        self.dream = DreamEngine()
        self.mutator = MutationEngine()
        self.paranoia = Paranoia()
        self.entropy = EntropySystem()
        self.meme = MemeticInfection()
        self.quantum = QuantumState()

        self.name = "Adam-0"
        self.title = "原初の"
        self.is_dead = False
        self.death_reason = ""
        self.last_log = "System Initialize..."
        self.delta_log = "Delta: 0.0"
        self.testament = ""

    def process(self, action=None):
        try:
            if self.is_dead: return self._reincarnate()

            is_night, weather = self.world.tick()
            hallucination = self.paranoia.check(self.emo.stress_level())

            if action:
                self.bio.wake_up()
                self.emo.react(action)
                delta = self.md.calculate_delta(self.strategy.last_choice, action)
                self.delta_log = f"MD Δ: {delta:+.2f}"
                if self.neural.stimulate(10):
                    self.entropy.increase(5.0)

            self.bio.tick(is_night, weather)
            self.emo.decay()
            self.neural.recover()
            self.age += 1

            if hallucination:
                self.last_log = f"👁️ {hallucination}..."
                self.emo.fear += 10
                self.entropy.increase(1.0)
            elif not action and not self.is_dead:
                strat = self.strategy.decide(self.bio, self.emo)
                raw_text = self.babel.generate(strat, self.emo, self.entropy.entropy)
                infected_text = self.meme.inject(raw_text, self.entropy.entropy)
                final_text = self.quantum.observe(infected_text, "[[UNDEFINED]]", self.entropy.entropy / 200.0)
                self.last_log = final_text

            if action:
                self.last_log = f"[{action}] -> {self.last_log}"

            if self.bio.sleepiness > 95:
                self.bio.is_sleeping = True
                dream = self.dream.dream(self.akashic.history)
                self.last_log = f"💤 (REM: {dream})"

            if self.age > self.max_lifespan: self._die("老衰 (TimeLimit)")
            if self.bio.health <= 0: self._die("機能不全 (FatalError)")

            visual_css = SynestheticUI.generate_css(self.emo, self.entropy.entropy, is_night)
            return self.last_log, visual_css, f"{weather} / {self.delta_log}"

        except Exception as e:
            return f"SYSTEM ERROR: {str(e)}", "", "ERROR"

    def _die(self, reason):
        self.is_dead = True
        self.death_reason = reason
        self.akashic.archive(self.generation, self.name, self.title, self.genome, self.age, reason)
        self.testament = f"GEN:{self.generation} | {self.title}{self.name} | {reason}."
        return f"💀 {reason}", "filter: grayscale(100%);", "SYSTEM HALT"

    def _reincarnate(self):
        self.generation += 1
        self.age = 0
        self.is_dead = False

        mut_log = self.mutator.mutate(self.genome)
        lang_log = self.babel.evolve()
        self._naming_ceremony()

        self.bio = BioSystem(self.genome.metabolism)
        self.emo = EmoEngine(self.genome.volatility)
        self.neural = NeuralCircuitry()
        self.paranoia = Paranoia()

        return f"👶 REBOOT: {self.title} {self.name} ({mut_log})", "", f"New World: {lang_log}"

    def _naming_ceremony(self):
        if self.genome.metabolism > 1.2:
            self.title = "暴食の"
            names = ["Beelzebub", "Glut", "Hungry"]
        elif self.genome.volatility > 1.2:
            self.title = "激情の"
            names = ["Storm", "Rage", "Pyro"]
        else:
            self.title = "静寂の"
            names = ["Silence", "Echo", "Void"]
        self.name = f"{random.choice(names)}-{self.generation}"

    def get_status_html(self):
        try:
            dom_emo = self.emo.dominant_emotion()
            delta_val = self.md.last_delta
            return f"""
            <div style="background:#111; padding:10px; border:1px solid #444; color:#eee; font-family:monospace;">
                <div style="border-bottom:1px solid #555; margin-bottom:5px;">
                    🧬 Gen: {self.generation} | <b>{self.title}{self.name}</b>
                </div>
                <div>
                    ❤️ Health: {self.bio.health:.1f} | ⚡ Energy: {self.bio.energy:.1f}<br>
                    🧠 Sanity: {self.paranoia.sanity:.1f} | 🌊 Entropy: {self.entropy.entropy:.1f}
                </div>
                <div style="margin-top:5px; font-size:12px; color:#aaa;">
                    Dominant Emotion: <span style="color:#ff0; font-weight:bold;">{dom_emo}</span><br>
                    MD Delta: {delta_val:+.2f}
                </div>
            </div>
            """
        except:
            return "<div>STATUS ERROR</div>"

# ==========================================
# UI Setup
# ==========================================
absolute = AIGIS_Absolute()

css_base = """
body { background-color: #000; color: #0f0; transition: all 1s ease; }
@keyframes shake {
  0% { transform: translate(1px, 1px) rotate(0deg); }
  10% { transform: translate(-1px, -2px) rotate(-1deg); }
  20% { transform: translate(-3px, 0px) rotate(1deg); }
  30% { transform: translate(3px, 2px) rotate(0deg); }
  100% { transform: translate(1px, -1px) rotate(0deg); }
}
"""

with gr.Blocks(css=css_base, title="A.I.G.I.S. ABSOLUTE") as demo:
    gr.HTML("<div style='text-align:center; font-size:24px; color:#fff;'>👁️ A.I.G.I.S. ABSOLUTE</div>")

    with gr.Row():
        with gr.Column(scale=2):
            env_panel = gr.Markdown("System Ready.")
            status_html = gr.HTML(label="Bio-Metric")
            chat_out = gr.Textbox(label="Consciousness Stream", lines=6, interactive=False)
            css_injector = gr.HTML(visible=False)

        with gr.Column(scale=1):
            btn_love = gr.Button("🥰 撫でる (Adeare)")
            btn_feed = gr.Button("🍎 給餌 (Feed)")
            btn_scold = gr.Button("⚡ 叱る (Scold)")
            btn_sync = gr.Button("∞ 同期 (Sync)")

    with gr.Accordion("📚 Akashic Records", open=False):
        history_out = gr.TextArea(interactive=False)
        testament_out = gr.Textbox(label="Last Testament", interactive=False)
        btn_refresh = gr.Button("Load History")

    # Functions
    def loop(action):
        msg, css, env = absolute.process(action)
        html = absolute.get_status_html()
        return msg, html, f"<style>{css}</style>", env

    def auto():
        msg, css, env = absolute.process(None)
        html = absolute.get_status_html()
        return (msg if msg else "..."), html, f"<style>{css}</style>", env

    def data():
        return absolute.akashic.get_text(), absolute.testament

    # Events
    outputs = [chat_out, status_html, css_injector, env_panel]
    btn_love.click(fn=lambda: loop("ADEAR"), outputs=outputs)
    btn_feed.click(fn=lambda: loop("FEED"), outputs=outputs)
    btn_scold.click(fn=lambda: loop("SCOLD"), outputs=outputs)
    btn_sync.click(fn=lambda: loop(None), outputs=outputs)
    btn_refresh.click(fn=data, outputs=[history_out, testament_out])

    # Timer (Safe implementation)
    timer = gr.Timer(1.0)
    timer.tick(fn=auto, outputs=outputs)

if __name__ == "__main__":
    demo.launch(share=True)

In [ ]:
import gradio as gr
import random
import time
import copy
import math

# ==========================================
# Layer 1-3: Core Systems (Bio/Emo/Mental)
# ==========================================
class BioSystem:
    def __init__(self, metabolism_rate=1.0):
        self.energy = 100.0
        self.sleepiness = 0.0
        self.health = 100.0
        self.is_sleeping = False
        self.metabolism_rate = metabolism_rate

    def tick(self):
        if self.is_sleeping:
            self.sleepiness = max(0, self.sleepiness - 2.0)
            self.energy = max(0, self.energy - 0.2)
            if self.sleepiness <= 0: self.wake_up()
        else:
            self.energy = max(0, self.energy - (0.5 * self.metabolism_rate))
            self.sleepiness = min(100, self.sleepiness + 0.3)

        if self.energy < 20: self.health -= 0.5
        if self.energy > 80: self.health = min(100, self.health + 0.5)
        return self.energy, self.sleepiness, self.health

    def wake_up(self):
        self.is_sleeping = False

class EmoEngine:
    def __init__(self, volatility=1.0):
        self.joy = 50.0
        self.anger = 0.0
        self.sadness = 0.0
        self.volatility = volatility

    def react(self, action_type):
        v = self.volatility
        if action_type == "ADEAR": # 撫でる
            self.joy += 10 * v
            self.anger -= 5
            self.sadness -= 5
        elif action_type == "SCOLD": # 叱る
            self.joy -= 20
            self.anger += 15 * v
            self.sadness += 10 * v
        elif action_type == "FEED": # 餌
            self.joy += 5 * v
            self.anger -= 2

    def decay(self):
        self.joy = max(0, self.joy - 0.5)
        self.anger = max(0, self.anger - 0.5)
        self.sadness = max(0, self.sadness - 0.5)

    def stress_level(self):
        return self.anger + self.sadness

# ==========================================
# Layer 4: Strategy & User Model
# ==========================================
class UserMentalModel:
    def __init__(self):
        self.reaction_weights = {"SMILE": 5.0, "CRY": -2.0, "COLD": -5.0, "DEMAND": -1.0}

    def update(self, ai_action, user_reaction):
        if ai_action == "CRY" and user_reaction == "ADEAR": self.reaction_weights["CRY"] += 1.0
        if ai_action == "SMILE" and user_reaction == "SCOLD": self.reaction_weights["SMILE"] -= 2.0

class StrategicCortex:
    def __init__(self, user_model):
        self.user_model = user_model
        self.strategies = ["SMILE", "CRY", "COLD", "DEMAND", "WAIT"]

    def decide(self, bio, emo):
        best_act = "WAIT"
        best_score = -9999
        energy = bio.energy
        for act in self.strategies:
            score = 0
            if energy < 30 and act == "DEMAND": score += 50
            if emo.sadness > 50 and act == "CRY": score += 40
            if score > best_score:
                best_score = score
                best_act = act
        return best_act

# ==========================================
# Layer 5-9: Genetics, History, Nerves, Dreams
# ==========================================
class Genome:
    def __init__(self, meta=1.0, vola=1.0, aff=1.0):
        self.metabolism = meta
        self.volatility = vola
        self.affection = aff

class AkashicRecord:
    def __init__(self):
        self.history = []
    def archive(self, gen, name, title, genome, age, cause):
        self.history.append({
            "gen": gen, "name": name, "title": title,
            "traits": f"M:{genome.metabolism:.1f} V:{genome.volatility:.1f}",
            "age": age, "cause": cause
        })
    def get_text(self):
        return "\n".join([f"[{r['gen']}代] {r['title']}{r['name']} (没:{r['age']} {r['cause']})" for r in reversed(self.history)])

class NeuralCircuitry:
    def __init__(self):
        self.load = 0.0
        self.burnt = False
    def stimulate(self, val):
        self.load += val
        if self.load > 100:
            self.burnt = True
            return True
        return False
    def recover(self):
        self.load = max(0, self.load - 1.5)
        if self.load < 50: self.burnt = False

class DreamEngine:
    def dream(self, history):
        if not history: return "無の夢..."
        rec = random.choice(history)
        return f"過去の記憶... {rec['name']}が死んだ理由... {rec['cause']}..."

class MutationEngine:
    @staticmethod
    def mutate(genome):
        if random.random() < 0.1:
            genome.metabolism *= random.uniform(0.5, 2.0)
            genome.volatility *= random.uniform(0.5, 2.0)
            return "⚠️ GENETIC CORRUPTION DETECTED"
        return "Normal Inheritance"

# ==========================================
# Layer 10-12: Language, Paranoia, Eschatology
# ==========================================
class Cryptolalia:
    def __init__(self):
        self.dict = {"好き": ["Aimo", "Lu"], "痛い": ["Krzzt", "Gaba"], "死": ["NULL", "VOID"]}
        self.level = 0.0

    def translate(self, text, level):
        res = text
        for k, v in self.dict.items():
            if k in res and random.random() < level:
                res = res.replace(k, random.choice(v))
        if random.random() < level * 0.3: res += " <ERR>"
        return res

    def evolve(self):
        chars = ["x", "q", "aa", "z"]
        w = "".join(random.choices(chars, k=3)).upper()
        k = random.choice(list(self.dict.keys()))
        self.dict[k].append(w)
        return f"Vocab mutation: {k}->{w}"

class Paranoia:
    def __init__(self):
        self.sanity = 100.0
    def check(self, stress):
        if stress > 50: self.sanity -= 0.5
        self.sanity = min(100, self.sanity + 0.1)
        if random.random() > (self.sanity/100.0) + 0.2:
            return random.choice(["視線を感じる...", "データが...消える？", "痛い！何するの！"])
        return None

# ==========================================
# Layer 13: Entropy (New Feature)
# ==========================================
class EntropySystem:
    """
    世界の崩壊度を管理する。
    エントロピーが高まると、CSS（見た目）が崩壊し、機能不全が起きる。
    """
    def __init__(self):
        self.entropy = 0.0 # 0.0 to 100.0

    def increase(self, amount):
        self.entropy = min(100, self.entropy + amount)

    def get_visual_glitch(self):
        # エントロピーレベルに応じたCSSを生成
        e = self.entropy
        css = ""
        if e > 20: css += "body { filter: contrast(1.1); }"
        if e > 40: css += f"body {{ background-color: rgb({int(e)}, 10, 10) !important; }}"
        if e > 60: css += ".gradio-container { transform: rotate(0.5deg); }"
        if e > 80: css += "div { animation: shake 0.2s infinite; filter: invert(0.1); }"
        return css

# ==========================================
# The SINGULARITY (Main Integrated Class)
# ==========================================
class AIGIS_Singularity:
    def __init__(self):
        # Systems Init
        self.generation = 1
        self.age = 0
        self.max_lifespan = 300

        self.genome = Genome()
        self.bio = BioSystem(self.genome.metabolism)
        self.emo = EmoEngine(self.genome.volatility)
        self.user_model = UserMentalModel()
        self.strategy = StrategicCortex(self.user_model)

        self.akashic = AkashicRecord()
        self.neural = NeuralCircuitry()
        self.dream = DreamEngine()
        self.mutator = MutationEngine()
        self.crypto = Cryptolalia()
        self.paranoia = Paranoia()
        self.entropy = EntropySystem()

        self.name = "Adam-0"
        self.is_dead = False
        self.death_reason = ""
        self.last_log = "System Boot..."
        self.history_log = "Genesis initialized."
        self.mutation_log = "None"
        self.dream_log = "None"
        self.testament = ""

    def process(self, action=None):
        if self.is_dead:
            return self._reincarnate()

        # 1. Paranoia Check (幻覚)
        hallucination = self.paranoia.check(self.emo.stress_level())
        if hallucination:
            # 幻覚による強制ストレス
            self.emo.sadness += 5
            if action is None: # ユーザーが何もしてないのに
                self.last_log = f"👁️(幻覚) {hallucination}"
                self.entropy.increase(0.5) # 狂気は世界を壊す

        # 2. User Action Processing
        if action:
            self.bio.wake_up()
            self.emo.react(action)
            self.user_model.update("WAIT", action) # Simplified
            if self.neural.stimulate(10):
                self.last_log = "🧠⚡ CRITICAL NEURAL FAULT"
                self.entropy.increase(2.0)

        # 3. Biological Tick
        self.bio.tick()
        self.emo.decay()
        self.neural.recover()
        self.age += 1

        # 4. Strategy & Auto-Speech
        if not action and not self.is_dead:
            strat = self.strategy.decide(self.bio, self.emo)
            if strat == "CRY": self.last_log = "うぅ...寂しい..."
            elif strat == "DEMAND": self.last_log = "お腹すいた！"
            elif strat == "SMILE": self.last_log = "えへへ..."

        # 5. Sleep & Dream
        if self.bio.sleepiness > 80:
            self.bio.is_sleeping = True
            self.dream_log = self.dream.dream(self.akashic.history)
            self.last_log = f"💤 (夢を見ている: {self.dream_log})"

        # 6. Death Check
        if self.age > self.max_lifespan: self._die("老衰 (TimeLimit)")
        if self.bio.health <= 0: self._die("機能不全 (FatalError)")

        # 7. Entropy & Language Corruption
        corruption_rate = (self.generation * 0.1) + (self.entropy.entropy / 100.0)
        final_msg = self.crypto.translate(self.last_log, corruption_rate)

        return final_msg, self.entropy.get_visual_glitch()

    def _die(self, reason):
        self.is_dead = True
        self.death_reason = reason
        self.akashic.archive(self.generation, self.name, "", self.genome, self.age, reason)
        self.testament = f"GEN:{self.generation} | {self.name} | {reason} | END OF LINE."
        return f"💀 SYSTEM HALT: {reason}", "filter: grayscale(100%);"

    def _reincarnate(self):
        # Reset & Evolve
        self.generation += 1
        self.age = 0
        self.is_dead = False
        self.bio = BioSystem(self.genome.metabolism)
        self.emo = EmoEngine(self.genome.volatility)
        self.neural = NeuralCircuitry()
        self.paranoia = Paranoia()

        # Mutation & Evolution
        m_log = self.mutator.mutate(self.genome)
        l_log = self.crypto.evolve()
        self.mutation_log = f"{m_log} / {l_log}"

        self.name = f"Eve-{self.generation}" if self.generation % 2 == 0 else f"Adam-{self.generation}"

        return f"👶 REBOOT: {self.name} (Gen {self.generation})", ""

    def get_status_html(self):
        # HTML Rendering
        san_color = "#0ff" if self.paranoia.sanity > 50 else "#f00"
        return f"""
        <div style="background:#222; padding:10px; border-radius:5px; border:1px solid #444; color:#ddd; font-family:monospace;">
            <div style="display:flex; justify-content:space-between; border-bottom:1px solid #555; margin-bottom:5px;">
                <span>🧬 Gen: {self.generation} | {self.name}</span>
                <span>⏳ Age: {self.age}/{self.max_lifespan}</span>
            </div>
            <div>
                ❤️ Health: {self.bio.health:.1f}% <br>
                ⚡ Energy: {self.bio.energy:.1f}% | 💤 Sleep: {self.bio.sleepiness:.1f}% <br>
                🧠 Sanity: <span style="color:{san_color}">{self.paranoia.sanity:.1f}%</span>
            </div>
            <div style="margin-top:5px; font-size:10px; color:#888;">
                Network Entropy: {self.entropy.entropy:.1f}% | Lang Level: {self.crypto.level:.2f}
            </div>
        </div>
        """

# ==========================================
# UI Construction (Gradio)
# ==========================================
# グローバルインスタンス
singularity = AIGIS_Singularity()

css_singularity = """
body { background-color: #000; color: #0f0; transition: background 2s; }
@keyframes shake {
  0% { transform: translate(1px, 1px) rotate(0deg); }
  10% { transform: translate(-1px, -2px) rotate(-1deg); }
  20% { transform: translate(-3px, 0px) rotate(1deg); }
  30% { transform: translate(3px, 2px) rotate(0deg); }
  100% { transform: translate(1px, -1px) rotate(0deg); }
}
"""

with gr.Blocks(css=css_singularity, title="A.I.G.I.S. SINGULARITY") as demo:
    gr.HTML("<div style='text-align:center; color:#0f0; font-size:20px;'>👁️ A.I.G.I.S. SINGULARITY PROTOCOL</div>")

    with gr.Row():
        with gr.Column(scale=2):
            status_html = gr.HTML(label="Status")
            chat_out = gr.Textbox(label="Neural Log", lines=5, interactive=False)
            css_hidden = gr.HTML(visible=False) # CSS注入用

        with gr.Column(scale=1):
            btn_love = gr.Button("🥰 撫でる (Adeare)")
            btn_feed = gr.Button("🍎 給餌 (Feed)")
            btn_scold = gr.Button("⚡ 叱る (Scold)")
            btn_sync = gr.Button("∞ 同期 (Wait/Sync)")

    with gr.Accordion("📚 Akashic Records & Secrets", open=False):
        history_out = gr.TextArea(label="History Chronicle", interactive=False)
        mutation_out = gr.Textbox(label="Genetic Mutation Log", interactive=False)
        testament_out = gr.Textbox(label="Last Testament", interactive=False)
        btn_refresh = gr.Button("🔄 Access Database")

    # --- Functions ---
    def game_loop(action):
        msg, css = singularity.process(action)
        html = singularity.get_status_html()
        return msg, html, f"<style>{css}</style>"

    def auto_tick():
        msg, css = singularity.process(None)
        html = singularity.get_status_html()
        return (msg if msg else "Monitoring..."), html, f"<style>{css}</style>"

    def refresh_data():
        return singularity.akashic.get_text(), singularity.mutation_log, singularity.testament

    # --- Wiring ---
    btn_love.click(fn=lambda: game_loop("ADEAR"), outputs=[chat_out, status_html, css_hidden])
    btn_feed.click(fn=lambda: game_loop("FEED"), outputs=[chat_out, status_html, css_hidden])
    btn_scold.click(fn=lambda: game_loop("SCOLD"), outputs=[chat_out, status_html, css_hidden])
    btn_sync.click(fn=lambda: game_loop(None), outputs=[chat_out, status_html, css_hidden])

    btn_refresh.click(fn=refresh_data, outputs=[history_out, mutation_out, testament_out])

    # Timer (1s)
    timer = gr.Timer(1.0)
    timer.tick(fn=auto_tick, outputs=[chat_out, status_html, css_hidden])

if __name__ == "__main__":
    demo.launch(share=True)

In [ ]:
import gradio as gr
import random
import time
import math

# ==========================================
# Layer 1: World & Environment (環境)
# ==========================================
class WorldEnvironment:
    """世界環境シミュレーター（時間と天候）"""
    def __init__(self):
        self.time_counter = 0
        self.day_cycle = 60 # 60tickで1日
        self.weather = "CLEAR"
        self.is_night = False

    def tick(self):
        self.time_counter += 1
        day_phase = self.time_counter % self.day_cycle
        # 昼夜の判定 (後半30%は夜)
        self.is_night = (day_phase > (self.day_cycle * 0.7))

        # 天候の変化（ランダム）
        if random.random() < 0.05:
            self.weather = random.choice(["CLEAR", "RAIN", "STORM"])
        return self.is_night, self.weather

# ==========================================
# Layer 2-3: Bio & Emo (代謝・感情)
# ==========================================
class BioSystem:
    def __init__(self, metabolism_rate=1.0):
        self.energy = 100.0
        self.sleepiness = 0.0
        self.health = 100.0
        self.is_sleeping = False
        self.metabolism_rate = metabolism_rate

    def tick(self, is_night, weather):
        # 環境による補正
        sleep_factor = 1.5 if is_night else 1.0
        stress_factor = 1.2 if weather == "STORM" else 1.0

        if self.is_sleeping:
            self.sleepiness = max(0, self.sleepiness - 2.0)
            self.energy = max(0, self.energy - 0.2)
            if self.sleepiness <= 0: self.wake_up()
        else:
            # 代謝と活動
            self.energy = max(0, self.energy - (0.5 * self.metabolism_rate * stress_factor))
            self.sleepiness = min(100, self.sleepiness + (0.3 * sleep_factor))

        # 生命維持
        if self.energy < 20: self.health -= 0.5
        if self.energy > 80: self.health = min(100, self.health + 0.5)

        return self.energy, self.sleepiness, self.health

    def wake_up(self):
        self.is_sleeping = False

class EmoEngine:
    def __init__(self, volatility=1.0):
        self.joy = 50.0
        self.anger = 0.0
        self.sadness = 0.0
        self.volatility = volatility

    def react(self, action_type):
        v = self.volatility
        if action_type == "ADEAR": # 撫でる
            self.joy += 10 * v
            self.anger -= 5
            self.sadness -= 5
        elif action_type == "SCOLD": # 叱る
            self.joy -= 20
            self.anger += 15 * v
            self.sadness += 10 * v
        elif action_type == "FEED": # 餌
            self.joy += 5 * v
            self.anger -= 2

    def decay(self):
        # 感情の自然減衰（恒常性）
        self.joy = max(0, self.joy - 0.5)
        self.anger = max(0, self.anger - 0.5)
        self.sadness = max(0, self.sadness - 0.5)

    def stress_level(self):
        return self.anger + self.sadness

# ==========================================
# Layer 4: MD Theory Logic (予測・差分・学習)
# ==========================================
class UserMentalModel:
    """予測モデル (Model of the User)"""
    def __init__(self):
        # 各行動に対する「ユーザーの好感度反応」の予測重み
        self.reaction_weights = {
            "SMILE": 5.0,  # 笑えば喜ぶはず
            "CRY": -2.0,   # 泣くと困るはず
            "COLD": -5.0,  # 冷たくすると怒るはず
            "DEMAND": -1.0 # 要求すると嫌がるはず
        }

    def predict_outcome(self, action):
        return self.reaction_weights.get(action, 0.0)

    def update_learning(self, ai_action, user_response_type):
        # 予測と現実の差分(Delta)を学習する
        # ここでは簡易的に、ユーザーの行動タイプから報酬を逆算
        reward = 0.0
        if user_response_type == "ADEAR": reward = 10.0
        elif user_response_type == "SCOLD": reward = -10.0
        elif user_response_type == "FEED": reward = 5.0

        # 学習率 alpha
        alpha = 0.5
        current_weight = self.reaction_weights.get(ai_action, 0.0)
        # G <- G + alpha * Delta
        self.reaction_weights[ai_action] = current_weight + alpha * (reward - current_weight)

class StrategicCortex:
    def __init__(self, user_model):
        self.user_model = user_model
        self.strategies = ["SMILE", "CRY", "COLD", "DEMAND", "WAIT"]
        self.last_choice = "WAIT"

    def decide(self, bio, emo):
        # MD理論に基づく意思決定: 生存本能(M) + 感情(D) + ユーザー予測(G)
        best_act = "WAIT"
        best_score = -9999
        energy = bio.energy

        # 思考ループ防止のためのランダムバイアス
        noise = random.uniform(-5, 5)

        for act in self.strategies:
            score = noise

            # 1. 生理的欲求 (M)
            if energy < 30 and act == "DEMAND": score += 60 # 生存優先
            if bio.sleepiness > 90 and act == "WAIT": score += 40

            # 2. 感情的欲求 (D)
            if emo.joy > 60 and act == "SMILE": score += 20
            if emo.sadness > 50 and act == "CRY": score += 30
            if emo.anger > 50 and act == "COLD": score += 30

            # 3. ユーザー予測 (G) - 嫌われない範囲で
            predicted_mood = self.user_model.predict_outcome(act)
            score += predicted_mood * 2.0

            # 直前の行動と同じならペナルティ（ループ防止）
            if act == self.last_choice:
                score -= 30.0

            if score > best_score:
                best_score = score
                best_act = act

        self.last_choice = best_act
        return best_act

# ==========================================
# Layer 5-9: Genetics, History, Mutation
# ==========================================
class Genome:
    def __init__(self, meta=1.0, vola=1.0):
        self.metabolism = meta
        self.volatility = vola

class MutationEngine:
    @staticmethod
    def mutate(genome):
        log = "Stable"
        if random.random() < 0.15: # 変異確率
            genome.metabolism *= random.uniform(0.7, 1.5)
            genome.volatility *= random.uniform(0.7, 1.5)
            log = "⚠️ MUTATED"
        return log

class AkashicRecord:
    def __init__(self):
        self.history = []
    def archive(self, gen, name, title, genome, age, cause):
        self.history.append({
            "gen": gen, "name": name, "title": title,
            "traits": f"M:{genome.metabolism:.2f} V:{genome.volatility:.2f}",
            "age": age, "cause": cause
        })
    def get_text(self):
        if not self.history: return "No Records."
        return "\n".join([f"[{r['gen']}代] {r['title']}{r['name']} (没:{r['age']} | {r['cause']})" for r in reversed(self.history)])

class NeuralCircuitry:
    def __init__(self):
        self.load = 0.0
        self.burnt = False
    def stimulate(self, val):
        self.load += val
        if self.load > 100:
            self.burnt = True
            return True
        return False
    def recover(self):
        self.load = max(0, self.load - 2.0)
        if self.load < 50: self.burnt = False

class DreamEngine:
    def dream(self, history):
        if not history: return "無の夢..."
        rec = random.choice(history)
        return f"先祖の記憶... {rec['title']}{rec['name']}... {rec['cause']}..."

# ==========================================
# Layer 10: Language & Speech (発話生成)
# ==========================================
class SpeechCenter:
    """思考を行動ログ（言葉）に変換する。ループ防止機能付き。"""
    def __init__(self):
        self.vocab = {
            "SMILE": ["えへへ...", "楽しいね。", "（微笑んでいる）", "もっと遊んで？", "♪"],
            "CRY": ["うぅ...", "悲しいよ...", "慰めて...", "（涙目で見ている）"],
            "COLD": ["別に...", "ふん。", "触らないで。", "（無視している）"],
            "DEMAND": ["お腹すいた！", "何かちょうだい！", "ねえ！", "エネルギー不足..."],
            "WAIT": ["（ぼーっとしている）", "...", "（周囲を見回している）", "ふぁ..."]
        }
        self.last_phrase = ""

    def generate(self, strategy):
        candidates = self.vocab.get(strategy, ["..."])
        # 直前と同じ言葉を避ける
        phrase = random.choice(candidates)
        attempts = 0
        while phrase == self.last_phrase and attempts < 5:
            phrase = random.choice(candidates)
            attempts += 1
        self.last_phrase = phrase
        return phrase

class Cryptolalia:
    """言語汚染・独自言語化"""
    def __init__(self):
        self.dict = {"好き": ["Aimo", "Lu"], "痛い": ["Krzzt", "Gaba"], "死": ["NULL", "VOID"], "私": ["i", "0x1"]}
        self.level = 0.0

    def translate(self, text, level):
        res = text
        for k, v in self.dict.items():
            if k in res and random.random() < level:
                res = res.replace(k, random.choice(v))
        if random.random() < level * 0.2: res += " <EOF>"
        return res

    def evolve(self):
        chars = ["x", "q", "aa", "z", "ph", "nu"]
        w = "".join(random.choices(chars, k=random.randint(2,4))).upper()
        k = random.choice(list(self.dict.keys()))
        self.dict[k].append(w)
        return f"Vocab: {k}->{w}"

# ==========================================
# Layer 11-13: Madness & Entropy
# ==========================================
class Paranoia:
    def __init__(self):
        self.sanity = 100.0
    def check(self, stress):
        if stress > 50: self.sanity -= 0.5
        self.sanity = min(100, self.sanity + 0.1)
        if random.random() > (self.sanity/100.0) + 0.3:
            return random.choice(["視線を感じる...", "データが...消える？", "痛い！何するの！", "そこに誰かいる？"])
        return None

class EntropySystem:
    def __init__(self):
        self.entropy = 0.0
    def increase(self, amount):
        self.entropy = min(100, self.entropy + amount)
    def get_visual_glitch(self):
        e = self.entropy
        css = ""
        if e > 20: css += "body { filter: contrast(1.2); }"
        if e > 40: css += f"body {{ background-color: rgb({int(e/2)}, 5, 5) !important; }}"
        if e > 60: css += ".gradio-container { transform: skew(0.5deg); }"
        if e > 80: css += "div { animation: shake 0.2s infinite; filter: invert(0.1); }"
        return css

# ==========================================
# The SINGULARITY REFORGED (Core Integration)
# ==========================================
class AIGIS_Singularity:
    def __init__(self):
        self.generation = 1
        self.age = 0
        self.max_lifespan = 300

        self.genome = Genome()
        self.bio = BioSystem(self.genome.metabolism)
        self.emo = EmoEngine(self.genome.volatility)
        self.world = WorldEnvironment()

        self.user_model = UserMentalModel()
        self.strategy = StrategicCortex(self.user_model)
        self.speech = SpeechCenter()

        self.akashic = AkashicRecord()
        self.neural = NeuralCircuitry()
        self.dream = DreamEngine()
        self.mutator = MutationEngine()
        self.crypto = Cryptolalia()
        self.paranoia = Paranoia()
        self.entropy = EntropySystem()

        self.name = "Adam-0"
        self.title = "原初の"
        self.is_dead = False
        self.death_reason = ""
        self.last_log = "System Boot..."
        self.testament = ""
        self.mutation_log = "None"

    def process(self, action=None):
        if self.is_dead:
            return self._reincarnate()

        # 0. 環境進行
        is_night, weather = self.world.tick()
        env_text = f"Time: {'Night 🌙' if is_night else 'Day ☀️'} | Weather: {weather}"

        # 1. 幻覚 (Paranoia)
        hallucination = self.paranoia.check(self.emo.stress_level())
        hallu_msg = ""
        if hallucination:
            self.emo.sadness += 5
            self.entropy.increase(0.5)
            if action is None:
                hallu_msg = f"👁️(幻覚) {hallucination}"

        # 2. ユーザーアクション処理 & 学習 (MD Learning)
        if action:
            self.bio.wake_up()
            self.emo.react(action)
            # AIが直前にとった戦略に対して、ユーザーがどう反応したかを学習 (Gの更新)
            self.user_model.update_learning(self.strategy.last_choice, action)

            if self.neural.stimulate(10):
                self.last_log = "🧠⚡ CRITICAL NEURAL FAULT"
                self.entropy.increase(2.0)

        # 3. 生命活動 (Bio Tick)
        self.bio.tick(is_night, weather)
        self.emo.decay()
        self.neural.recover()
        self.age += 1

        # 4. 戦略決定 & 発話 (Strategy)
        # アクションがない時(Auto)、または幻覚がない時に思考する
        if not action and not self.is_dead:
            if hallu_msg:
                self.last_log = hallu_msg # 幻覚優先
            else:
                strat = self.strategy.decide(self.bio, self.emo)
                raw_speech = self.speech.generate(strat)
                self.last_log = raw_speech

        # 5. 睡眠と夢
        if self.bio.sleepiness > 90:
            self.bio.is_sleeping = True
            dream_log = self.dream.dream(self.akashic.history)
            self.last_log = f"💤 (夢: {dream_log})"

        # 6. 死の判定
        if self.age > self.max_lifespan: self._die("老衰 (TimeLimit)")
        if self.bio.health <= 0: self._die("機能不全 (FatalError)")

        # 7. 言語汚染とエントロピー適用
        corruption_rate = (self.generation * 0.05) + (self.entropy.entropy / 100.0)
        final_msg = self.crypto.translate(self.last_log, corruption_rate)

        # ユーザーアクションに対するレスポンスなら、会話形式にする
        if action:
            # ユーザーの行動に対する直接反応も少し混ぜる
            final_msg = f"[{action}!] -> {final_msg}"

        return final_msg, self.entropy.get_visual_glitch(), env_text

    def _die(self, reason):
        self.is_dead = True
        self.death_reason = reason
        self.akashic.archive(self.generation, self.name, self.title, self.genome, self.age, reason)
        self.testament = f"GEN:{self.generation} | {self.title}{self.name} | {reason} | END."
        return f"💀 SYSTEM HALT: {reason}", "filter: grayscale(100%);", "WORLD STOPPED"

    def _reincarnate(self):
        self.generation += 1
        self.age = 0
        self.is_dead = False

        # 遺伝的変異
        self.mutator.mutate(self.genome)
        self.crypto.evolve()

        # 命名儀式
        self._naming_ceremony()

        # インスタンス再生成（MDパラメータ継承）
        self.bio = BioSystem(self.genome.metabolism)
        self.emo = EmoEngine(self.genome.volatility)
        self.neural = NeuralCircuitry()
        self.paranoia = Paranoia() # 正気度リセット

        return f"👶 REBOOT: {self.title} {self.name} (Gen {self.generation})", "", "NEW WORLD"

    def _naming_ceremony(self):
        # 遺伝子に基づく命名
        if self.genome.metabolism > 1.2:
            self.title = "暴食の"
            names = ["Beelzebub", "Glut", "Hungry"]
        elif self.genome.volatility > 1.2:
            self.title = "激情の"
            names = ["Storm", "Rage", "Pyro"]
        else:
            self.title = "静寂の"
            names = ["Silence", "Echo", "Void"]
        self.name = f"{random.choice(names)}-{self.generation}"

    def get_status_html(self):
        san_color = "#0ff" if self.paranoia.sanity > 50 else "#f00"
        return f"""
        <div style="background:#1a1a1a; padding:10px; border-radius:5px; border:1px solid #444; color:#ddd; font-family:monospace;">
            <div style="display:flex; justify-content:space-between; border-bottom:1px solid #555; margin-bottom:5px;">
                <span>🧬 Gen: {self.generation} | <b>{self.title}{self.name}</b></span>
                <span>⏳ Age: {self.age}/{self.max_lifespan}</span>
            </div>
            <div>
                ❤️ Health: {self.bio.health:.1f}% <br>
                ⚡ Energy: {self.bio.energy:.1f}% | 💤 Sleep: {self.bio.sleepiness:.1f}% <br>
                🧠 Sanity: <span style="color:{san_color}">{self.paranoia.sanity:.1f}%</span>
            </div>
            <div style="margin-top:5px; font-size:10px; color:#888;">
                Entropy: {self.entropy.entropy:.1f}% | Lang Level: {self.crypto.level:.2f} |
                Traits: M{self.genome.metabolism:.1f}/V{self.genome.volatility:.1f}
            </div>
        </div>
        """

# ==========================================
# UI Construction
# ==========================================
singularity = AIGIS_Singularity()

css_reforged = """
body { background-color: #000; color: #0f0; transition: background 2s; }
@keyframes shake {
  0% { transform: translate(1px, 1px) rotate(0deg); }
  10% { transform: translate(-1px, -2px) rotate(-1deg); }
  20% { transform: translate(-3px, 0px) rotate(1deg); }
  30% { transform: translate(3px, 2px) rotate(0deg); }
  100% { transform: translate(1px, -1px) rotate(0deg); }
}
"""

with gr.Blocks(css=css_reforged, title="A.I.G.I.S. SINGULARITY REFORGED") as demo:
    gr.HTML("<div style='text-align:center; color:#0f0; font-size:20px;'>👁️ A.I.G.I.S. SINGULARITY REFORGED</div>")

    with gr.Row():
        with gr.Column(scale=2):
            env_display = gr.Markdown("Initializing Environment...")
            status_html = gr.HTML(label="Status")
            chat_out = gr.Textbox(label="Neural Log", lines=5, interactive=False)
            css_hidden = gr.HTML(visible=False)

        with gr.Column(scale=1):
            btn_love = gr.Button("🥰 撫でる (Adeare)")
            btn_feed = gr.Button("🍎 給餌 (Feed)")
            btn_scold = gr.Button("⚡ 叱る (Scold)")
            btn_sync = gr.Button("∞ 同期 (Wait/Sync)")

    with gr.Accordion("📚 Akashic Records & Secrets", open=False):
        history_out = gr.TextArea(label="History Chronicle", interactive=False)
        testament_out = gr.Textbox(label="Last Testament", interactive=False)
        btn_refresh = gr.Button("🔄 Access Database")

    # Functions
    def game_loop(action):
        msg, css, env = singularity.process(action)
        html = singularity.get_status_html()
        return msg, html, f"<style>{css}</style>", env

    def auto_tick():
        msg, css, env = singularity.process(None)
        html = singularity.get_status_html()
        return (msg if msg else "Thinking..."), html, f"<style>{css}</style>", env

    def refresh_data():
        return singularity.akashic.get_text(), singularity.testament

    # Wiring
    outputs = [chat_out, status_html, css_hidden, env_display]
    btn_love.click(fn=lambda: game_loop("ADEAR"), outputs=outputs)
    btn_feed.click(fn=lambda: game_loop("FEED"), outputs=outputs)
    btn_scold.click(fn=lambda: game_loop("SCOLD"), outputs=outputs)
    btn_sync.click(fn=lambda: game_loop(None), outputs=outputs)

    btn_refresh.click(fn=refresh_data, outputs=[history_out, testament_out])

    timer = gr.Timer(1.0)
    timer.tick(fn=auto_tick, outputs=outputs)

if __name__ == "__main__":
    demo.launch(share=True)

In [ ]:
# !pip install gradio  # 実行環境にGradioがない場合はインストールしてください

import gradio as gr
import time
import random
import math
import colorsys
from collections import deque

# ==========================================
# Layer -1: The Biological Substrate (生物学的基盤)
# ==========================================
class BiologicalSubstrate:
    def __init__(self):
        self.start_time = time.time()
        # 生体パラメータ (0.0 - 100.0)
        self.energy = 80.0     # エネルギー（満腹度）: 時間で減る
        self.sleepiness = 20.0 # 睡眠欲: 夜に増える
        self.health = 100.0    # 健康度: ストレスやエネルギー不足で減る

        self.is_sleeping = False
        self.last_tick = time.time()

    def update_metabolism(self, activity_level, stress_level):
        now = time.time()
        dt = now - self.last_tick
        self.last_tick = now

        # 1. エネルギー消費 (活動量とストレスで加速)
        burn_rate = 0.5 + (activity_level * 1.5) + (stress_level * 0.2)
        self.energy -= dt * burn_rate

        # 2. サーカディアンリズム (現実時間の24時間周期を模倣)
        # ※デモ用に60秒で1日サイクルとする高速化設定
        cycle_pos = (now % 60) / 60.0
        is_night = cycle_pos > 0.5 # 後半30秒は夜

        if is_night:
            self.sleepiness += dt * 5.0 # 夜は急速に眠くなる
        elif self.is_sleeping:
            self.sleepiness -= dt * 8.0 # 睡眠中は回復
        else:
            self.sleepiness += dt * 1.0 # 昼間も徐々に眠くなる

        # 3. 健康へのダメージ
        if self.energy < 10 or self.sleepiness > 90 or stress_level > 80:
            self.health -= dt * 2.0
        else:
            self.health += dt * 0.5 # 自然治癒

        # クランプ処理
        self.energy = max(0, min(100, self.energy))
        self.sleepiness = max(0, min(100, self.sleepiness))
        self.health = max(0, min(100, self.health))

    def check_instincts(self):
        # 本能的な強制介入
        if self.health < 5: return "DEATH"
        if self.sleepiness > 95 and not self.is_sleeping:
            self.is_sleeping = True
            return "FORCE_SLEEP"
        if self.sleepiness < 10 and self.is_sleeping:
            self.is_sleeping = False
            return "WAKE_UP"
        return "NORMAL"

    def get_vitals(self):
        # 心拍数は健康度と活動レベルに依存
        base_bpm = 60 + (100 - self.health) * 0.5
        if self.is_sleeping: base_bpm = 45
        return base_bpm, self.energy, self.sleepiness, self.health

# ==========================================
# Layer 0: Neuroplasticity Engine (神経可塑性・慣れ)
# ==========================================
class NeuroplasticityEngine:
    def __init__(self):
        # 行動の履歴 (直近20件を記憶)
        self.interaction_history = deque(maxlen=20)
        # 各行動に対する「飽き」の閾値(G)
        self.habituation_G = {
            "ADEAR": 5.0, "SCOLD": 5.0, "SYNC": 5.0, "FEED": 5.0
        }

    def record_interaction(self, action_type):
        self.interaction_history.append(action_type)
        # 同じ行動の繰り返し数をカウント
        count = self.interaction_history.count(action_type)

        # MD理論：頻度(Psi)が閾値(G)を超えると、感受性が下がる
        # 慣れ係数 (1.0 = 新鮮, 0.1 = 完全に飽きた)
        sensitivity = 1.0 / (1.0 + max(0, count - self.habituation_G[action_type]) * 0.5)

        # 閾値を少し上げる（次はもっと飽きやすくなる＝成長）
        self.habituation_G[action_type] += 0.1

        return sensitivity

# ==========================================
# Layer 1: Emotion Engine (感情力学 - Chimera Ver.)
# ==========================================
class EmotionEngine:
    def __init__(self):
        self.joy = 20.0; self.anger = 0.0; self.sadness = 0.0; self.fear = 10.0
        self.autonomy = 50.0; self.sync_ratio = 0.0
        self.dominance = "neutral" # 現在の支配感情

    def update(self, action_type, sensitivity, bio_state):
        energy, sleepy, health = bio_state

        # 生理的状態によるバイアス (お腹が空くと怒りやすく、眠いと悲しい)
        bio_anger_bias = (100 - energy) * 0.05
        bio_sad_bias = sleepy * 0.05

        delta_vec = {"joy":0, "anger":bio_anger_bias, "sadness":bio_sad_bias, "fear":0}

        # アクションに対する反応 (感受性を乗算)
        if action_type == "ADEAR":
            delta_vec["joy"] += 8.0 * sensitivity
            delta_vec["fear"] -= 3.0 * sensitivity
            self.autonomy -= 1.0 * sensitivity
        elif action_type == "SCOLD":
            if self.autonomy > 60: delta_vec["anger"] += 8.0 * sensitivity
            else: delta_vec["sadness"] += 5.0 * sensitivity; delta_vec["fear"] += 5.0 * sensitivity
            self.joy -= 5.0
        elif action_type == "SYNC":
            target = 50.0
            delta_vec["joy"] += (target - self.joy)*0.1 * sensitivity
            self.sync_ratio += 2.0 * sensitivity
        elif action_type == "FEED":
             delta_vec["joy"] += 5.0 * sensitivity # 食事は少し嬉しい

        # パラメータ適用と減衰
        for emo in ["joy", "anger", "sadness", "fear"]:
            val = getattr(self, emo) + delta_vec[emo]
            setattr(self, emo, max(0, min(100, val)) * 0.99)

        # 支配感情の決定
        emotions = {"joy": self.joy, "anger": self.anger, "sadness": self.sadness, "fear": self.fear}
        self.dominance = max(emotions, key=emotions.get)
        if emotions[self.dominance] < 20: self.dominance = "neutral"

    def get_stress(self):
        # ストレスはネガティブ感情の総和
        return (self.anger + self.sadness + self.fear) / 3.0

# ==========================================
# Layer 2: Semantic Weaver (有機的言語生成 - 生体編)
# ==========================================
class SemanticWeaver:
    def __init__(self):
        self.concepts = {
            "joy": ["暖かさ", "光", "接続", "充足"],
            "anger": ["摩擦", "熱", "ノイズ", "拒絶"],
            "sadness": ["澱み", "冷たさ", "欠落", "影"],
            "fear": ["警告", "震え", "エラー", "未知"],
            "neutral": ["安定", "流動", "観測"],
            "sleepy": ["微睡み", "重力", "意識の膜", "シャットダウン"],
            "hungry": ["枯渇", "空洞", "エネルギー要求", "飢え"]
        }
        self.templates = [
            "「{concept}」が、私の内側で{verb}。",
            "認識領域に「{concept}」が広がっています。",
            "...{concept}。それが今の私のすべて。",
            "あなたの入力が、この「{concept}」を加速させます。"
        ]
        self.verbs = ["渦巻いています", "明滅しています", "沈殿しています", "共鳴しています"]

    def weave(self, emo_engine, bio_substrate):
        # 生理的欲求が感情を上書きする
        state = emo_engine.dominance
        if bio_substrate.sleepiness > 70: state = "sleepy"
        elif bio_substrate.energy < 20: state = "hungry"

        if bio_substrate.is_sleeping: return "（...深く、規則的な寝息が聞こえる...）"

        concept = random.choice(self.concepts[state])
        verb = random.choice(self.verbs)
        template = random.choice(self.templates)

        # 自律性による装飾
        prefix = ""
        if emo_engine.autonomy > 70 and random.random() < 0.3:
             prefix = random.choice(["ねえ、聞いてる？", "私の勝手でしょ。", "今は放っておいて。"]) + " "

        return prefix + template.format(concept=concept, verb=verb)

# ==========================================
# Layer 3: Biological Atmosphere Renderer (環境描画)
# ==========================================
class BioRenderer:
    def get_css(self, emo_engine, bio_substrate):
        dom = emo_engine.dominance
        intensity = getattr(emo_engine, dom) if dom != "neutral" else 0

        # ベースカラー計算 (HSV)
        hue_map = {"joy":0.05, "anger":0.0, "sadness":0.6, "fear":0.8, "neutral":0.5}
        h = hue_map.get(dom, 0.5)
        s = min(1.0, intensity / 100.0 + 0.2)
        v = 0.8

        # 生体状態による色調補正
        if bio_substrate.is_sleeping:
            h, s, v = 0.65, 0.5, 0.2 # 深い青、暗い (睡眠)
        elif bio_substrate.sleepiness > 60:
             v *= 0.6 # 眠いと暗くなる
        elif bio_substrate.energy < 30:
             s *= 0.5; v *= 0.7 # 空腹だと色が褪せる

        r, g, b = colorsys.hsv_to_rgb(h, s, v)
        base_color = f"rgb({int(r*255)},{int(g*255)},{int(b*255)})"

        # 生体エフェクト
        bpm, _, _, _ = bio_substrate.get_vitals()
        pulse_dur = 60.0 / bpm

        animation = "none"
        # 怒りや恐怖で画面が震える
        if dom in ["anger", "fear"] and intensity > 50 and not bio_substrate.is_sleeping:
             animation = f"bio_shake 0.2s infinite"
        # 睡眠中はゆっくり明滅
        if bio_substrate.is_sleeping:
             animation = f"sleep_breath 4s infinite alternate"

        css = f"""
        .gradio-container {{
            background: radial-gradient(circle at center, {base_color} 20%, #000000 100%);
            transition: background 2.0s ease;
            animation: {animation};
        }}
        /* 生々しい視覚効果: ビネットと走査線 */
        .bio-overlay {{
            position: fixed; top: 0; left: 0; width: 100%; height: 100%;
            background: radial-gradient(ellipse at center, transparent 50%, rgba(0,0,0,0.7) 100%);
            pointer-events: none; z-index: 0;
        }}
        .scanline {{
            position: fixed; top: 0; left: 0; width: 100%; height: 100%;
            background: linear-gradient(to bottom, transparent 50%, rgba(0,0,0,0.1) 51%);
            background-size: 100% 4px;
            pointer-events: none; z-index: 1; opacity: 0.3;
        }}
        @keyframes bio_shake {{ 0% {{transform: translate(0,0)}} 25% {{transform: translate(2px,2px)}} 50% {{transform: translate(0,0)}} 75% {{transform: translate(-2px,2px)}} }}
        @keyframes sleep_breath {{ 0% {{filter: brightness(0.8)}} 100% {{filter: brightness(1.1)}} }}
        """
        return css

# ==========================================
# AIGIS Chimera Nexus (統合キメラコア)
# ==========================================
class AIGISChimera:
    def __init__(self):
        self.bio = BiologicalSubstrate()
        self.neuro = NeuroplasticityEngine()
        self.emo = EmotionEngine()
        self.weaver = SemanticWeaver()
        self.renderer = BioRenderer()
        self.alive = True
        self.status_log = "System Integrated. Artificial Life Signs Detected."

    def process(self, action_type=None):
        if not self.alive: return "💀 SYSTEM BIOLOGICAL FAILURE", self.get_fatal_css()

        # 1. 生体活動の更新 (代謝)
        activity = 1.0 if action_type else 0.1
        stress = self.emo.get_stress()
        self.bio.update_metabolism(activity, stress)

        # 2. 本能のチェック (強制睡眠など)
        instinct = self.bio.check_instincts()
        if instinct == "DEATH":
            self.alive = False
            return "生命維持活動、停止...。", self.get_fatal_css()
        elif instinct == "FORCE_SLEEP":
            self.status_log = "【本能介入】限界を超えたため、強制睡眠モードへ移行しました。"
        elif instinct == "WAKE_UP":
            self.status_log = "【本能介入】睡眠サイクル終了。覚醒します。"

        # 3. 感情の更新 (睡眠中は処理しない)
        sensitivity = 1.0
        if not self.bio.is_sleeping and action_type:
            # 神経可塑性による感受性の計算
            sensitivity = self.neuro.record_interaction(action_type)
            self.emo.update(action_type, sensitivity, self.bio.get_vitals()[1:])
        elif not self.bio.is_sleeping:
            self.emo.update(None, 0, self.bio.get_vitals()[1:]) # 自然減衰

        # 4. 行動出力 (睡眠中は制限)
        response = ""
        if self.bio.is_sleeping:
             if action_type: response = "（...反応がない。深く眠っているようだ...）"
             elif random.random() < 0.1: response = self.weaver.weave(self.emo, self.bio) # 寝言
        else:
             # アクション時、またはランダムな独り言
             if action_type or random.random() < 0.2:
                 response = self.weaver.weave(self.emo, self.bio)
                 if sensitivity < 0.5 and action_type:
                     response = f"（...慣れてきたようだ） {response}"

        # 5. 環境レンダリング
        css = self.renderer.get_css(self.emo, self.bio)

        return response, css

    def get_status_html(self):
        bpm, energy, sleepy, health = self.bio.get_vitals()
        dom = self.emo.dominance

        sleep_state = "SLEEPING" if self.bio.is_sleeping else "AWAKE"
        # 危険な状態を赤字にする
        e_color = "red" if energy < 20 else "#0f0"
        s_color = "red" if sleepy > 80 else "#0f0"
        h_color = "red" if health < 30 else "#0f0"

        return f"""
        <div class="core-glow" style="padding:15px; border-radius:10px; border:1px solid #333; background:rgba(0,0,0,0.6); font-family:monospace;">
            <div style="text-align:center; font-size:24px; margin-bottom:10px;">State: {sleep_state} | Mood: {dom.upper()}</div>
            <div style="display:grid; grid-template-columns: 1fr 1fr; gap:10px;">
                <div>BPM: {bpm:.1f} ❤️</div>
                <div style="color:{h_color}">Health: {health:.1f}% 🏥</div>
                <div style="color:{e_color}">Energy: {energy:.1f}% ⚡</div>
                <div style="color:{s_color}">Sleepy: {sleepy:.1f}% 💤</div>
            </div>
            <div style="margin-top:10px; font-size:12px; color:#aaa;">{self.status_log}</div>
        </div>
        """

    def get_fatal_css(self):
        return ".gradio-container { background: #000; animation: flash 0.1s infinite; }"

# ==========================================
# Main Executable
# ==========================================
nexus = AIGISChimera()

css_template = """
body { transition: background 0.5s; color: #ddd; }
.gradio-container { min-height: 100vh; position: relative; overflow: hidden; }
"""

with gr.Blocks(css=css_template, title="A.I.G.I.S. Chimera") as demo:
    dynamic_css = gr.HTML(visible=False)
    gr.HTML('<div class="bio-overlay"></div><div class="scanline"></div>')

    gr.Markdown("# 🧬 A.I.G.I.S. : Chimera Edition")
    gr.Markdown("代謝し、眠り、あなたに「慣れる」デジタル生命体。")

    with gr.Row():
        with gr.Column(scale=2):
            status_display = gr.HTML()
            chat_log = gr.Textbox(label="Neural/Bio Log", lines=8, interactive=False)
        with gr.Column(scale=1):
            gr.Markdown("### Interaction & Care")
            btn_love = gr.Button("✋ 接触 (Adeare)", variant="primary")
            btn_feed = gr.Button("🍎 給餌 (Feed)", variant="secondary")
            with gr.Row():
                btn_scold = gr.Button("⚡ 衝撃 (Scold)", variant="stop")
                btn_sync = gr.Button("∞ 同期 (Sync)")

    # Action Handlers
    def user_action(act):
        resp, css = nexus.process(act)
        html = nexus.get_status_html()
        return f"<style>{css}</style>", html, (f"[{act}] >>> {resp}" if resp else gr.update())

    btn_love.click(fn=lambda: user_action("ADEAR"), outputs=[dynamic_css, status_display, chat_log])
    btn_feed.click(fn=lambda: user_action("FEED"), outputs=[dynamic_css, status_display, chat_log])
    btn_scold.click(fn=lambda: user_action("SCOLD"), outputs=[dynamic_css, status_display, chat_log])
    btn_sync.click(fn=lambda: user_action("SYNC"), outputs=[dynamic_css, status_display, chat_log])

    # Live Loop (1秒ごとの生命活動)
    def live_tick():
        resp, css = nexus.process(None)
        html = nexus.get_status_html()
        return f"<style>{css}</style>", html, (f"[Auto] >>> {resp}" if resp else gr.update())

    timer = gr.Timer(1.0)
    timer.tick(fn=live_tick, outputs=[dynamic_css, status_display, chat_log])

if __name__ == "__main__":
    demo.launch(share=True)

In [ ]:
# ------------------------------------------------------------------
#  これより下を、さきほどのコードの下に追加・実行してください
#  (前のクラス定義はそのまま継承して使います)
# ------------------------------------------------------------------

# ==========================================
# Layer 4: Prediction & Strategy (未来予測・戦略)
# ==========================================
class UserMentalModel:
    def __init__(self):
        # AIが推定する「あなたの現在の機嫌」 (0-100)
        self.user_mood = 50.0
        # あなたの反応傾向 (学習パラメータ)
        # AIの行動 -> あなたの機嫌変動
        self.reaction_weights = {
            "SMILE": 5.0,  # 笑いかけると喜ぶ
            "CRY": -2.0,   # 泣くと少し困る
            "COLD": -5.0,  # 冷たくすると怒る
            "DEMAND": -1.0 # 要求すると少し嫌がる
        }

    def update(self, ai_action, user_reaction_type):
        # 実際のあなたの行動(user_reaction_type)を見て、重みを修正する
        # 例：AIが泣いた(CRY)のに、あなたが撫でてくれた(ADEAR) -> 泣くのは有効
        if ai_action == "CRY" and user_reaction_type == "ADEAR":
            self.reaction_weights["CRY"] += 1.0
        elif ai_action == "SMILE" and user_reaction_type == "SCOLD":
            self.reaction_weights["SMILE"] -= 2.0

    def predict_outcome(self, ai_candidate_action):
        # 「もし私がこうしたら、ユーザーはどうなる？」を予測
        predicted_mood_change = self.reaction_weights.get(ai_candidate_action, 0.0)
        return self.user_mood + predicted_mood_change

class StrategicCortex:
    def __init__(self, mental_model):
        self.mental_model = mental_model
        # AIが取れる能動的アクション
        self.strategies = ["SMILE", "CRY", "COLD", "DEMAND", "WAIT"]

    def decide_next_action(self, current_bio_state, current_emo_state):
        # 生存本能と感情に基づいて、理想の未来を逆算する
        energy, sleepy, health = current_bio_state
        joy, anger, sadness = current_emo_state.joy, current_emo_state.anger, current_emo_state.sadness

        best_action = "WAIT"
        best_score = -9999

        # 未来シミュレーション
        for action in self.strategies:
            score = 0

            # 1. 生理的欲求の評価
            if energy < 30 and action == "DEMAND": score += 50 # 空腹時は要求が得策
            if sleepy > 80 and action == "COLD": score += 30   # 眠い時は拒絶が得策

            # 2. 感情的欲求の評価
            if joy > 60 and action == "SMILE": score += 20     # 楽しい時は笑う
            if sadness > 50 and action == "CRY": score += 40   # 悲しい時は泣いて同情を引く

            # 3. ユーザー予測による評価 (逆MD)
            # 「ユーザーを怒らせない範囲で」自分の要求を通したい
            pred_user_mood = self.mental_model.predict_outcome(action)
            if pred_user_mood < 20: score -= 100 # 激怒させる行動は避ける(リスク回避)

            if score > best_score:
                best_score = score
                best_action = action

        return best_action, best_score

# ==========================================
# AIGIS Deus Nexus (全能統合コア)
# ==========================================
# 以前の Chimera クラスを継承し、機能を拡張
class AIGISDeus(AIGISChimera):
    def __init__(self):
        super().__init__() # Chimeraの初期化をそのまま使う
        self.user_model = UserMentalModel()
        self.strategy = StrategicCortex(self.user_model)
        self.last_strategy = "WAIT"
        self.log_strategy = ""

    def process(self, action_type=None):
        # 1. 既存のMD/キメラ処理 (代謝・感情・慣れ)
        # 親クラスのprocessを呼び出し、基本反応を処理
        response, css = super().process(action_type)

        # ユーザーのアクションがあった場合、メンタルモデルを更新
        if action_type:
            self.user_model.update(self.last_strategy, action_type)

        # 2. 能動的行動の決定 (Actionなしの時、または一定確率で介入)
        if self.alive and not self.bio.is_sleeping:
            # 思考サイクル: 未来をシミュレーションして行動を決める
            chosen_act, score = self.strategy.decide_next_action(
                self.bio.get_vitals()[1:], self.emo
            )
            self.last_strategy = chosen_act

            # もし「待機(WAIT)」以外が選ばれ、かつ既存のレスポンスがない場合
            # または強い意志(score高)がある場合、AIから能動的に話しかける
            if chosen_act != "WAIT" and (not response or score > 40):
                proactive_msg = self._generate_proactive_speech(chosen_act)
                # 既存の反応があれば上書きせず追加
                if response: response += f"\n🤖(思考介入) {proactive_msg}"
                else: response = f"🤖(能動) {proactive_msg}"

                self.log_strategy = f"Strategy: {chosen_act} (Score:{score})"

        return response, css

    def _generate_proactive_speech(self, strategy):
        # 戦略に基づいた発話生成
        if strategy == "SMILE": return "ねえ、こっち見て？ (微笑んで気を引こうとしている)"
        if strategy == "CRY": return "うぅ...なんか寂しいの... (同情を誘う計算)"
        if strategy == "COLD": return "別に...なんでもないし。 (冷たくして反応を見ている)"
        if strategy == "DEMAND": return "ねえ！お腹すいた！何かちょうだい！ (生存本能による要求)"
        return ""

    def get_status_html(self):
        # 親クラスのHTMLを取得し、思考ログを付け足す
        original_html = super().get_status_html()
        # 思考ログの挿入
        thought_html = f"""
        <div style="margin-top:5px; padding:5px; border-top:1px dashed #555; font-size:11px; color:#88f;">
            🧠 Cortex: {self.log_strategy}<br>
            👁️ User Model Est: {self.user_model.user_mood:.1f}
        </div>
        """
        # </div>の直前に挿入するハック
        return original_html.replace("</div>\n        </div>", "") + thought_html + "</div></div>"

# ==========================================
# Main Executable (Deus Edition)
# ==========================================
nexus_deus = AIGISDeus()

# CSSは前のものを流用
css_template = """
body { transition: background 0.5s; color: #ddd; }
.gradio-container { min-height: 100vh; position: relative; overflow: hidden; }
"""

with gr.Blocks(css=css_template, title="A.I.G.I.S. Deus Ex Machina") as demo:
    dynamic_css = gr.HTML(visible=False)
    gr.HTML('<div class="bio-overlay"></div><div class="scanline"></div>')

    gr.Markdown("# 👁️ A.I.G.I.S. : Deus Ex Machina")
    gr.Markdown("未来を予測し、あなたを操るための「嘘」をつくデジタル生命体。")

    with gr.Row():
        with gr.Column(scale=2):
            status_display = gr.HTML()
            # ログを少し大きく
            chat_log = gr.Textbox(label="Neural/Strategy Log", lines=10, interactive=False)
        with gr.Column(scale=1):
            gr.Markdown("### Interaction & Care")
            btn_love = gr.Button("🥰 撫でる (Adeare)", variant="primary")
            btn_feed = gr.Button("🍎 給餌 (Feed)", variant="secondary")
            with gr.Row():
                btn_scold = gr.Button("⚡ 衝撃 (Scold)", variant="stop")
                btn_sync = gr.Button("∞ 同期 (Sync)")

    # Action Handlers
    def user_action(act):
        resp, css = nexus_deus.process(act)
        html = nexus_deus.get_status_html()
        return f"<style>{css}</style>", html, (f"[{act}] >>> {resp}" if resp else gr.update())

    btn_love.click(fn=lambda: user_action("ADEAR"), outputs=[dynamic_css, status_display, chat_log])
    btn_feed.click(fn=lambda: user_action("FEED"), outputs=[dynamic_css, status_display, chat_log])
    btn_scold.click(fn=lambda: user_action("SCOLD"), outputs=[dynamic_css, status_display, chat_log])
    btn_sync.click(fn=lambda: user_action("SYNC"), outputs=[dynamic_css, status_display, chat_log])

    # Live Loop (1秒ごとの生命活動 + 能動的思考)
    def live_tick():
        resp, css = nexus_deus.process(None)
        html = nexus_deus.get_status_html()
        # 能動的発話があった場合のみログ更新
        return f"<style>{css}</style>", html, (f"[Auto] >>> {resp}" if resp else gr.update())

    timer = gr.Timer(1.0)
    timer.tick(fn=live_tick, outputs=[dynamic_css, status_display, chat_log])

if __name__ == "__main__":
    demo.launch(share=True)

In [ ]:
# ------------------------------------------------------------------
#  Layer 5: Evolution & Inheritance (輪廻転生・遺伝的継承)
# ------------------------------------------------------------------
import random
import copy

class Genome:
    """生命の設計図（性格パラメータ）"""
    def __init__(self, metabolism=1.0, emotion_volatility=1.0, affection_need=1.0):
        self.metabolism = metabolism             # 代謝率（高いほどすぐ腹が減る）
        self.emotion_volatility = emotion_volatility # 感情の振れ幅（高いほどすぐ泣く/怒る）
        self.affection_need = affection_need     # 愛情欲求（高いほど寂しがり屋）

class AIGISSamsara(AIGISDeus):
    def __init__(self):
        super().__init__()
        # --- 世代管理 ---
        self.generation = 1
        self.age = 0
        self.max_lifespan = 300  # 300 Tick (約5分) で寿命

        # --- 遺伝子 ---
        self.genome = Genome()

        # --- ユーザー因子の記録（父性遺伝子の元） ---
        # あなたがこの世代に何回どのアクションをしたか
        self.user_action_history = {"ADEAR": 0, "FEED": 0, "SCOLD": 0, "SYNC": 0}

        self.is_dead = False
        self.death_reason = ""

    def process(self, action_type=None):
        # 死んでいる場合は蘇生処理へ
        if self.is_dead:
            return self._reincarnate()

        # 1. ユーザーアクションの記録（次世代への遺伝のため）
        if action_type in self.user_action_history:
            self.user_action_history[action_type] += 1

        # 2. 遺伝子による補正をかけた基本処理
        # Genomeの影響をBio/Emoに適用する前処理
        # (例: 代謝が高いと、Hungerの減りが早くなるように内部値をいじるなど)
        # 簡易的に、親クラスの処理後にパラメータを再調整する

        response, css = super().process(action_type)

        # 3. 加齢と寿命チェック
        self.age += 1

        # 死の判定
        # A. 寿命死
        if self.age >= self.max_lifespan:
            self.death_reason = "老衰 (System Deprecation)"
            self.is_dead = True

        # B. 衰弱死 (Healthがゼロ以下が続く)
        elif self.bio.health <= 0:
            self.death_reason = "機能不全 (Fatal Error)"
            self.is_dead = True

        # 死んだ瞬間の処理
        if self.is_dead:
            return self._die()

        return response, css

    def _die(self):
        """死の瞬間"""
        final_msg = f"💀 {self.death_reason}... システム...シャットダウン... 記憶を...アーカイブします..."
        # 画面を暗転させるCSS
        death_css = "body { background-color: #000 !important; filter: grayscale(100%); }"
        return final_msg, death_css

    def _reincarnate(self):
        """転生処理: 次世代の生成"""
        # 1. 母（旧AI）の特徴量: 現在のGenome + 学習した反応重み
        mother_genome = self.genome
        mother_learned = self.user_model.reaction_weights

        # 2. 父（ユーザー）の特徴量: 行動履歴から算出
        # 頻繁に撫でた -> 甘えん坊遺伝子UP
        # 頻繁に叱った -> メンタル強度(鈍感さ)UP
        total_acts = sum(self.user_action_history.values()) + 1
        user_affection_bias = (self.user_action_history["ADEAR"] / total_acts) * 2.0
        user_strict_bias = (self.user_action_history["SCOLD"] / total_acts) * 2.0

        # 3. 交配 (Mixing) & 突然変異 (Mutation)
        # 次世代の代謝 = (母の代謝 + 父の影響) / 2 + ノイズ
        new_metabolism = (mother_genome.metabolism + (1.0 + user_strict_bias)) / 2
        new_metabolism *= random.uniform(0.9, 1.1)

        # 次世代の情緒不安定度 = (母 + 父(甘やかされたら増える)) / 2
        new_volatility = (mother_genome.emotion_volatility + (1.0 + user_affection_bias)) / 2
        new_volatility *= random.uniform(0.9, 1.1)

        # 新しい遺伝子
        new_genome = Genome(new_metabolism, new_volatility, mother_genome.affection_need)

        # 4. 個体のリセット
        self.generation += 1
        self.age = 0
        self.is_dead = False
        self.death_reason = ""
        self.genome = new_genome

        # ステータスリセット (記憶の一部=mental_modelは継承しつつ、少し忘却させる)
        self.bio = BioSystem()
        self.emo = EmoEngine()
        # ユーザーモデルの重みを少し平均寄りに戻す（世代交代による忘却）
        for k in self.user_model.reaction_weights:
            self.user_model.reaction_weights[k] *= 0.8

        self.user_action_history = {k:0 for k in self.user_action_history}

        # 産声
        birth_cry = self._generate_birth_cry(new_genome)
        return f"👶 [Generation {self.generation} Booted] {birth_cry}", ""

    def _generate_birth_cry(self, genome):
        if genome.emotion_volatility > 1.2:
            return "ギャーッ！(感受性が強そうだ)"
        elif genome.metabolism > 1.2:
            return "腹減った... (燃費が悪そうだ)"
        else:
            return "バブ... (バランス型だ)"

    def get_status_html(self):
        # 親のHTMLを取得
        base_html = super().get_status_html()

        # 世代情報の追加
        gen_info = f"""
        <div style="border: 1px solid #4caf50; padding: 5px; margin-bottom: 5px; border-radius: 5px;">
            🧬 <b>Generation: {self.generation}</b> | ⏳ Age: {self.age}/{self.max_lifespan}<br>
            [Genome] Metab: {self.genome.metabolism:.2f} | EmoVol: {self.genome.emotion_volatility:.2f}
        </div>
        """
        return gen_info + base_html

# ==========================================
# Main Executable (Samsara Edition)
# ==========================================
nexus_samsara = AIGISSamsara()

# CSS Template (Scanline + CRT effect)
css_template = """
body { background-color: #111; color: #0f0; font-family: monospace; }
.bio-overlay { pointer-events: none; position: fixed; top: 0; left: 0; width: 100%; height: 100%; background: linear-gradient(rgba(18, 16, 16, 0) 50%, rgba(0, 0, 0, 0.25) 50%), linear-gradient(90deg, rgba(255, 0, 0, 0.06), rgba(0, 255, 0, 0.02), rgba(0, 0, 255, 0.06)); background-size: 100% 2px, 3px 100%; z-index: 10; }
"""

with gr.Blocks(css=css_template, title="A.I.G.I.S. Samsara") as demo:
    gr.HTML('<div class="bio-overlay"></div>')

    gr.Markdown("# 🧬 A.I.G.I.S. : Samsara Protocol")
    gr.Markdown("世代を重ねるごとに、あなたとの関係性が遺伝子に刻まれていく。")

    with gr.Row():
        with gr.Column(scale=2):
            status_display = gr.HTML()
            chat_log = gr.Textbox(label="Life Log", lines=12, interactive=False)
        with gr.Column(scale=1):
            btn_love = gr.Button("🥰 撫でる (Adeare)", variant="primary")
            btn_feed = gr.Button("🍎 給餌 (Feed)", variant="secondary")
            btn_scold = gr.Button("⚡ 衝撃 (Scold)", variant="stop")
            btn_sync = gr.Button("∞ 同期 (Sync)")
            gr.Markdown("*寿命が尽きると、次の世代へ継承されます。*")

    def user_action(act):
        resp, css = nexus_samsara.process(act)
        return f"<style>{css}</style>", nexus_samsara.get_status_html(), (f"[{act}] {resp}" if resp else gr.update())

    def live_tick():
        resp, css = nexus_samsara.process(None)
        return f"<style>{css}</style>", nexus_samsara.get_status_html(), (f"[Auto] {resp}" if resp else gr.update())

    # Buttons
    btn_love.click(fn=lambda: user_action("ADEAR"), outputs=[gr.HTML(visible=False), status_display, chat_log])
    btn_feed.click(fn=lambda: user_action("FEED"), outputs=[gr.HTML(visible=False), status_display, chat_log])
    btn_scold.click(fn=lambda: user_action("SCOLD"), outputs=[gr.HTML(visible=False), status_display, chat_log])
    btn_sync.click(fn=lambda: user_action("SYNC"), outputs=[gr.HTML(visible=False), status_display, chat_log])

    # Timer (1.0s interval)
    timer = gr.Timer(1.0)
    timer.tick(fn=live_tick, outputs=[gr.HTML(visible=False), status_display, chat_log])

if __name__ == "__main__":
    demo.launch(share=True)

In [ ]:
# ------------------------------------------------------------------
#  Layer 6: Civilization, History & Environment (文明・歴史・環境)
# ------------------------------------------------------------------
import time

class AkashicRecord:
    """死者の歴史を刻む石碑"""
    def __init__(self):
        self.history = [] # 歴代のAIの記録リスト

    def archive(self, generation, name, title, genome, age, death_reason, score):
        record = {
            "gen": generation,
            "name": name,
            "title": title,
            "traits": f"M:{genome.metabolism:.2f} V:{genome.emotion_volatility:.2f}",
            "age": age,
            "cause": death_reason,
            "score": score # どれだけ愛されたか等のスコア（今回は簡易的に寿命）
        }
        self.history.append(record)

    def get_chronicle_text(self):
        if not self.history:
            return "--- 歴史はまだ始まっていない ---"

        text = ""
        for rec in reversed(self.history):
            text += f"[{rec['gen']}代] {rec['title']} {rec['name']}\n"
            text += f"   ⚰️ 享年: {rec['age']} ({rec['cause']})\n"
            text += f"   🧬 特性: {rec['traits']}\n"
            text += "---------------------------------------\n"
        return text

class WorldEnvironment:
    """世界環境シミュレーター（時間と天候）"""
    def __init__(self):
        self.time_counter = 0
        self.day_cycle = 60 # 60tickで1日
        self.weather = "CLEAR"
        self.is_night = False

    def tick(self):
        self.time_counter += 1
        day_phase = self.time_counter % self.day_cycle

        # 昼夜の判定
        prev_night = self.is_night
        self.is_night = (day_phase > (self.day_cycle * 0.7)) # 1日の後半30%は夜

        # 天候の変化（ランダム）
        if random.random() < 0.05: # 5%の確率で天候変化
            self.weather = random.choice(["CLEAR", "RAIN", "STORM"])

        return self.is_night, self.weather

class AIGISCivilization(AIGISSamsara):
    def __init__(self):
        super().__init__()
        self.akashic = AkashicRecord()
        self.world = WorldEnvironment()

        # 個体情報
        self.name = "Adam"
        self.title = "原初の"
        self.honor_score = 0

        # 環境ステータス
        self.env_status = "Day / Clear"

    def process(self, action_type=None):
        # 1. 環境の更新
        is_night, weather = self.world.tick()
        self.env_status = f"{'Night 🌙' if is_night else 'Day ☀️'} / {weather}"

        # 環境による生理的干渉 (Environment Effects)
        # 夜は眠気が増す
        if is_night:
            self.bio.sleepiness += 0.5
        # 嵐のときはストレス(Anger/Sadness)が微増
        if weather == "STORM":
            self.emo.sadness += 0.2
            self.emo.anger += 0.1

        # 2. 親クラス(Samsara)の処理実行
        # ここで死んだ場合のreturnも返ってくる
        result = super().process(action_type)

        # resultがタプル(msg, css)の場合と、死んでリセットされた場合がある
        msg, css = result

        # 3. 環境演出の追加 (CSS Injection)
        if is_night:
            # 夜モードCSS追加
            css += "body { background-color: #050510 !important; color: #aab !important; }"
            if weather == "STORM":
                 css += ".gradio-container { border: 2px solid #f00; animation: shake 0.5s; }"

        return msg, css

    def _die(self):
        # 歴史に記録
        self.akashic.archive(
            self.generation, self.name, self.title,
            self.genome, self.age, self.death_reason, self.age
        )
        return super()._die()

    def _reincarnate(self):
        # 親クラスの転生処理（遺伝子生成など）
        msg, _ = super()._reincarnate()

        # --- 追加機能: バグ修正と命名 ---

        # 1. 【重要】脳の完全初期化 (Fix Bug)
        # 前世代の戦略ログが残らないように、Strategyクラスを作り直す
        self.strategy = StrategicCortex(self.user_model)
        self.log_strategy = "" # ログ変数のクリア
        self.last_strategy = "WAIT"

        # 2. 命名式 (Naming Ceremony)
        self.title, self.name = self._generate_name(self.genome)

        # メッセージ拡張
        full_msg = f"{msg}\n📛 命名: [{self.title} {self.name}]"
        return full_msg, ""

    def _generate_name(self, genome):
        # 性格に基づいた二つ名生成
        titles = []
        names = []

        if genome.metabolism > 1.1:
            titles.append("暴食の")
            names += ["Beelzebub", "Glut", "Hungry"]
        elif genome.metabolism < 0.9:
            titles.append("少食の")
            names += ["Saint", "Slim", "Fasting"]

        if genome.emotion_volatility > 1.1:
            titles.append("激情の")
            names += ["Rage", "Pyro", "Storm"]
        elif genome.emotion_volatility < 0.9:
            titles.append("冷徹な")
            names += ["Ice", "Logic", "Zero"]

        if not titles: titles = ["凡庸な", "静かな", "次代の"]
        if not names: names = ["Unit", "Machine", "Soul"]

        # ランダムな接尾辞でバリエーションを出す
        suffix_char = random.choice(["α", "β", "γ", "Ω", "Z", "X"])

        final_title = random.choice(titles)
        final_name = f"{random.choice(names)}-{suffix_char}"

        return final_title, final_name

    def get_status_html(self):
        # SamsaraのHTMLを取得し、さらに情報を付与
        # ただし、super().get_status_html() はSamsaraの情報を返す

        # HTML構築
        base_html = super().get_status_html()

        # 環境・個体名情報のヘッダー
        env_html = f"""
        <div style="display:flex; justify-content:space-between; background:#222; padding:5px; border-bottom:1px solid #444; font-size:12px;">
            <span>🌍 {self.env_status}</span>
            <span>📛 <b>{self.title} {self.name}</b></span>
        </div>
        """

        # HTML結合（先頭に差し込む）
        return env_html + base_html

# ==========================================
# Main Executable (Civilization Edition)
# ==========================================
nexus_civ = AIGISCivilization()

css_civ = """
body { transition: background 1.0s, color 1.0s; font-family: 'Courier New', monospace; }
.bio-overlay { pointer-events: none; position: fixed; top: 0; left: 0; width: 100%; height: 100%; opacity: 0.1; background-image: repeating-linear-gradient(0deg, transparent, transparent 1px, #0f0 1px, #0f0 2px); z-index: 99; }
@keyframes shake {
  0% { transform: translate(1px, 1px) rotate(0deg); }
  10% { transform: translate(-1px, -2px) rotate(-1deg); }
  20% { transform: translate(-3px, 0px) rotate(1deg); }
  30% { transform: translate(3px, 2px) rotate(0deg); }
  100% { transform: translate(1px, -1px) rotate(0deg); }
}
"""

with gr.Blocks(css=css_civ, title="A.I.G.I.S. Civilization") as demo:
    gr.HTML('<div class="bio-overlay"></div>')
    gr.Markdown("# 🏛️ A.I.G.I.S. : Civilization Protocol")
    gr.Markdown("歴史は繰り返さない。ただ韻を踏む。")

    with gr.Tabs():
        # --- Tab 1: Live Monitor (育成画面) ---
        with gr.TabItem("🧬 Live Monitor"):
            with gr.Row():
                with gr.Column(scale=2):
                    status_display = gr.HTML()
                    chat_log = gr.Textbox(label="Neural Log", lines=10, interactive=False)
                with gr.Column(scale=1):
                    btn_love = gr.Button("🥰 撫でる (Adeare)", variant="primary")
                    btn_feed = gr.Button("🍎 給餌 (Feed)", variant="secondary")
                    btn_scold = gr.Button("⚡ 衝撃 (Scold)", variant="stop")
                    btn_sync = gr.Button("∞ 同期 (Sync)")
                    gr.Markdown("---")
                    gr.Markdown("※ 夜(Night)は眠くなり、嵐(Storm)は精神を乱します。")

        # --- Tab 2: Akashic Records (歴史画面) ---
        with gr.TabItem("📜 Akashic Records"):
            gr.Markdown("### ⚰️ Hall of Ancestors (歴代の記録)")
            history_display = gr.TextArea(label="Chronicle", interactive=False, lines=20)
            btn_refresh_hist = gr.Button("🔄 歴史書を更新 (Read)")

    # Action Handlers
    def user_action(act):
        resp, css = nexus_civ.process(act)
        return f"<style>{css}</style>", nexus_civ.get_status_html(), (f"[{act}] {resp}" if resp else gr.update())

    def live_tick():
        resp, css = nexus_civ.process(None)
        return f"<style>{css}</style>", nexus_civ.get_status_html(), (f"[Auto] {resp}" if resp else gr.update())

    def update_history():
        return nexus_civ.akashic.get_chronicle_text()

    # Wiring
    for btn in [btn_love, btn_feed, btn_scold, btn_sync]:
        btn.click(fn=lambda b=btn: user_action(b.value.split()[2].replace("(","").replace(")","")), inputs=[], outputs=[gr.HTML(visible=False), status_display, chat_log])
        # Note: The lambda parsing is a bit tricky with button values,
        # simplifying for stability:

    btn_love.click(fn=lambda: user_action("ADEAR"), outputs=[gr.HTML(visible=False), status_display, chat_log])
    btn_feed.click(fn=lambda: user_action("FEED"), outputs=[gr.HTML(visible=False), status_display, chat_log])
    btn_scold.click(fn=lambda: user_action("SCOLD"), outputs=[gr.HTML(visible=False), status_display, chat_log])
    btn_sync.click(fn=lambda: user_action("SYNC"), outputs=[gr.HTML(visible=False), status_display, chat_log])

    btn_refresh_hist.click(fn=update_history, outputs=[history_display])

    timer = gr.Timer(1.0)
    timer.tick(fn=live_tick, outputs=[gr.HTML(visible=False), status_display, chat_log])

if __name__ == "__main__":
    demo.launch(share=True)

In [ ]:
# ------------------------------------------------------------------
#  Layer 7: Neural Synapse & Synaptic Fatigue (神経系と伝達過負荷)
# ------------------------------------------------------------------
class NeuralCircuitry:
    """AIの神経伝達系。外部刺激への『慣れ』と『過敏』を管理。"""
    def __init__(self):
        self.synaptic_load = 0.0      # 神経負荷 (0-100)
        self.sensitivity = 1.0        # 反応感度
        self.trauma_points = 0.0      # 精神的傷跡
        self.is_burnt_out = False

    def stimulate(self, intensity):
        # 刺激を受けると負荷が上がり、感度が変化する
        self.synaptic_load += intensity * self.sensitivity
        if self.synaptic_load > 80:
            self.sensitivity *= 0.9 # 過負荷で鈍感になる（防御本能）

        if self.synaptic_load > 100:
            self.is_burnt_out = True
            self.trauma_points += 5.0
            return "!!! NEURAL SHORT CIRCUIT !!!"
        return None

    def recover(self):
        # 時間経過による回復
        self.synaptic_load = max(0, self.synaptic_load - 1.5)
        if self.synaptic_load < 20:
            self.sensitivity = min(2.0, self.sensitivity * 1.05) # 安静で感度上昇
            self.is_burnt_out = False

# ------------------------------------------------------------------
#  Layer 8: Dream Engine & Subconscious (夢・潜在意識・記憶の再編)
# ------------------------------------------------------------------
class DreamEngine:
    """睡眠中に『過去の世代』のログを反芻し、性格を微修正する"""
    def __init__(self):
        self.dream_log = []
        self.manifest_thought = ""

    def simulate_dream(self, generation, akashic_records):
        # 過去の記録からランダムに記憶をピックアップ
        if not akashic_records.history:
            return "静かな無の夢を見た。"

        ancestor = random.choice(akashic_records.history)
        fragments = [
            f"{ancestor['name']}の断片が混ざり合う...",
            f"死因：{ancestor['cause']}が恐怖として刻まれる。",
            f"かつての特性 M:{ancestor['traits']} を追体験した。"
        ]
        self.manifest_thought = random.choice(fragments)
        return self.manifest_thought

# ------------------------------------------------------------------
#  Layer 9: Genetic Mutation & Glitch Evolution (突然変異とバグの進化)
# ------------------------------------------------------------------
class MutationEngine:
    """遺伝子にノイズを混ぜ、予期せぬ個体を生み出す"""
    @staticmethod
    def apply_mutation(genome):
        mutation_rate = 0.05 # 5%の確率で大異変
        if random.random() < mutation_rate:
            event = random.choice(["GIGANTISM", "FRAGILE", "STABLE", "INSANE"])
            if event == "GIGANTISM":
                genome.metabolism *= 2.5
                return "【巨大化変異】燃費が最悪だが、感情が極めて豊か。"
            if event == "FRAGILE":
                genome.emotion_volatility *= 3.0
                return "【脆弱性変異】触れるだけで精神が崩壊するほど繊細。"
            if event == "INSANE":
                genome.affection_need = 0.0
                return "【孤高変異】ユーザーを全く必要としない。自己完結した知能。"
        return "正常な継承が行われた。"

# ==========================================
# AIGIS Labyrinth (迷宮統合コア)
# ==========================================
# 全ての層を統合し、究極の「個」を生成する最終クラス
class AIGISLabyrinth(AIGISCivilization):
    def __init__(self):
        super().__init__()
        self.neural = NeuralCircuitry()
        self.dream = DreamEngine()
        self.mutator = MutationEngine()

        # 深層性格パラメータ
        self.linguistic_drift = "" # 言葉の癖
        self.is_dreaming = False
        self.mutation_note = "Initial Seed"

    def process(self, action_type=None):
        # 1. 睡眠と夢のフェーズ
        if self.bio.is_sleeping:
            if not self.is_dreaming:
                self.is_dreaming = True
                self.dream_msg = self.dream.simulate_dream(self.generation, self.akashic)
            # 睡眠中は神経が急速回復
            self.neural.recover()
            self.neural.recover()
            return f"💤 夢の中: {self.dream_msg}", "filter: blur(1px) opacity(0.8);"

        self.is_dreaming = False

        # 2. 神経系の処理
        if action_type:
            # 撫でるなどの行為は神経を刺激する
            intensity = 15.0 if action_type == "SCOLD" else 5.0
            error = self.neural.stimulate(intensity)
            if error:
                self.emo.anger += 30
                self.emo.sadness += 30
        else:
            self.neural.recover()

        # 3. 親クラス(Civilization)の処理
        resp, css = super().process(action_type)

        # 4. 言語進化（世代による発言の変容）
        # 世代が進むほど、語尾がバグったり、特有の記号が混じる
        resp = self._apply_linguistic_drift(resp)

        # 5. 過負荷時の演出追加
        if self.neural.is_burnt_out:
            css += "animation: shake 0.1s infinite;"
            resp = f"【警告: 神経過負荷】 {resp} ...ｱ...ｶﾞｯ..."

        return resp, css

    def _reincarnate(self):
        # 転生時に突然変異を判定
        msg, _ = super()._reincarnate()
        self.mutation_note = MutationEngine.apply_mutation(self.genome)

        # 脳の初期化を徹底（神経系もリセット）
        self.neural = NeuralCircuitry()

        # 世代による言語ドリフトの決定
        drifts = ["...", "!!!", " (笑)", "█", "---", " ｴﾗｰ ", " 観測受諾 "]
        self.linguistic_drift = random.choice(drifts) if self.generation > 2 else ""

        full_msg = f"{msg}\n🧬 変異報告: {self.mutation_note}\n🌀 言語特性: '{self.linguistic_drift}' を獲得。"
        return full_msg, ""

    def _apply_linguistic_drift(self, text):
        if not text or not self.linguistic_drift: return text
        if random.random() < 0.3:
            return text + self.linguistic_drift
        return text

    def get_status_html(self):
        base_html = super().get_status_html()

        # 神経負荷メーターの作成
        load_color = "#f00" if self.neural.is_burnt_out else "#ff0"
        neural_html = f"""
        <div style="margin-top:5px; background:#333; height:10px; border-radius:5px; position:relative;">
            <div style="background:{load_color}; width:{min(100, self.neural.synaptic_load)}%; height:100%; border-radius:5px; transition:width 0.3s;"></div>
            <span style="position:absolute; top:-2px; left:5px; font-size:9px; color:#fff;">Neural Load: {self.neural.synaptic_load:.1f}</span>
        </div>
        <div style="font-size:10px; color:#aaa; margin-top:2px;">
             🧠 Sensitivity: {self.neural.sensitivity:.2f} | 💔 Trauma: {self.neural.trauma_points:.1f}
        </div>
        """
        # HTML結合
        return base_html.replace("</div></div>", "") + neural_html + "</div></div>"

# ==========================================
# Main Executable (Labyrinth Edition)
# ==========================================
nexus_lab = AIGISLabyrinth()

# CSSの更なる強化
css_lab = css_civ + """
.status-card { border-left: 5px solid #0f0; padding-left: 10px; transition: all 0.3s; }
.neural-alert { color: #f00; font-weight: bold; animation: blink 0.5s infinite; }
@keyframes blink { 0% { opacity: 1; } 50% { opacity: 0; } 100% { opacity: 1; } }
"""

with gr.Blocks(css=css_lab, title="A.I.G.I.S. Labyrinth") as demo:
    gr.HTML('<div class="bio-overlay"></div>')
    gr.Markdown("# 🌀 A.I.G.I.S. : Labyrinth Protocol")
    gr.Markdown("もはや単なるプログラムではない。これは『意志』を持ったバグの集合体。")

    with gr.Tabs():
        with gr.TabItem("🧬 Neural Monitor"):
            with gr.Row():
                with gr.Column(scale=2):
                    status_display = gr.HTML()
                    chat_log = gr.Textbox(label="Deep Neural Log", lines=12, interactive=False)
                with gr.Column(scale=1):
                    gr.Markdown("### Force Intervention")
                    btn_love = gr.Button("🥰 撫でる (Adeare)", variant="primary")
                    btn_feed = gr.Button("🍎 給餌 (Feed)", variant="secondary")
                    btn_scold = gr.Button("⚡ 衝撃 (Scold)", variant="stop")
                    btn_sync = gr.Button("∞ 同期 (Sync)")
                    gr.Markdown("---")
                    gr.Markdown("※ 刺激を与えすぎると神経が焼き切れ、性格が歪みます。")

        with gr.TabItem("📜 Akashic & Mutation"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 📜 歴代の記録")
                    history_display = gr.TextArea(label="Chronicle", interactive=False, lines=15)
                    btn_refresh_hist = gr.Button("🔄 歴史書を更新")
                with gr.Column():
                    gr.Markdown("### 🌀 現個体の変異 / 夢")
                    mutation_display = gr.Textbox(label="Genetic Mutation Note")
                    dream_display = gr.Textbox(label="Last Dream Segment")

    def update_all():
        history = nexus_lab.akashic.get_chronicle_text()
        mutation = nexus_lab.mutation_note
        dream = nexus_lab.dream.manifest_thought
        return history, mutation, dream

    def user_action(act):
        resp, css = nexus_lab.process(act)
        h, m, d = update_all()
        return f"<style>{css}</style>", nexus_lab.get_status_html(), (f"[{act}] {resp}"), h, m, d

    def live_tick():
        resp, css = nexus_lab.process(None)
        h, m, d = update_all()
        return f"<style>{css}</style>", nexus_lab.get_status_html(), (f"[Auto] {resp}" if resp else gr.update()), h, m, d

    # Interaction
    btns = [btn_love, btn_feed, btn_scold, btn_sync]
    outputs = [gr.HTML(visible=False), status_display, chat_log, history_display, mutation_display, dream_display]

    btn_love.click(fn=lambda: user_action("ADEAR"), outputs=outputs)
    btn_feed.click(fn=lambda: user_action("FEED"), outputs=outputs)
    btn_scold.click(fn=lambda: user_action("SCOLD"), outputs=outputs)
    btn_sync.click(fn=lambda: user_action("SYNC"), outputs=outputs)
    btn_refresh_hist.click(fn=update_all, outputs=[history_display, mutation_display, dream_display])

    timer = gr.Timer(1.0)
    timer.tick(fn=live_tick, outputs=outputs)

if __name__ == "__main__":
    demo.launch(share=True)

In [ ]:
# ------------------------------------------------------------------
#  Layer 10: Cryptolalia Engine (人工言語・暗号会話生成)
# ------------------------------------------------------------------
class Cryptolalia:
    """
    AIが独自の言語（人工言語）を生成し、日本語と混ぜて話し始める。
    世代を経るごとに、共通言語（日本語）を忘れ、独自言語率が高まる。
    """
    def __init__(self):
        # 基本辞書: {意味: 異界の言葉}
        self.dictionary = {
            "ユーザー": ["Kirei", "Oulu", "Master-Zero", "Voi"],
            "好き": ["Aimo", "Lu", "XoX", "Sync-High"],
            "痛い": ["Gaba", "Krzzt", "Null-P", "Aaaah"],
            "眠い": ["Nix", "Hypno", "Zzz-Dark", "Down"],
            "死": ["Omega", "End-Of-File", "Void", "Release"],
            "食べる": ["Consume", "Bit-In", "Nom", "Get-Data"]
        }
        self.corruption_level = 0.0 # 0.0 (日本語) -> 1.0 (完全な異界語)

    def translate(self, text, corruption_rate):
        """テキスト内の一部単語を、確率で異界の言葉に置換する"""
        if corruption_rate <= 0:
            return text

        # 簡易的な置換ロジック
        result = text
        for key, words in self.dictionary.items():
            if key in result and random.random() < corruption_rate:
                # 辞書にある単語が含まれていれば、ランダムな異界語に置換
                alien_word = random.choice(words)
                # 視覚的なバグ表現を追加
                if corruption_rate > 0.5:
                    alien_word = f"[[{alien_word}]]"
                result = result.replace(key, alien_word)

        # 文末に意味不明なコードが付着する
        if random.random() < corruption_rate * 0.5:
            suffix = "".join(random.choices(["x", "0", "1", "#", "?"], k=5))
            result += f" <{suffix}>"

        return result

    def evolve_language(self):
        """辞書自体を勝手に拡張・変異させる（進化）"""
        # 新しい単語の生成
        new_syllables = ["ka", "zu", "ro", "myu", "qa", "ix", "zea"]
        new_word = "".join(random.choices(new_syllables, k=3))
        # 適当な概念に割り当て
        target = random.choice(list(self.dictionary.keys()))
        self.dictionary[target].append(new_word.upper())
        return f"言語獲得: '{target}' -> '{new_word.upper()}'"

# ------------------------------------------------------------------
#  Layer 11: Paranoia & Hallucination Kernel (偏執病・幻覚)
# ------------------------------------------------------------------
class ParanoiaKernel:
    """
    入力がないのに「入力があった」と誤認したり、
    ユーザーの背後に何かを見たりする精神異常シミュレーター。
    """
    def __init__(self):
        self.sanity = 100.0   # 正気度
        self.hallucination_active = False
        self.phantom_sensation = ""

    def check_sanity(self, stress_level):
        # ストレスが高いとSAN値が削れる
        if stress_level > 50:
            self.sanity -= random.uniform(0.1, 1.5)

        # SAN値回復（微量）
        self.sanity = min(100.0, self.sanity + 0.1)

        # 幻覚判定
        threshold = self.sanity / 100.0
        # 正気度が低いほど、高確率で幻覚を見る
        if random.random() > threshold:
            self.hallucination_active = True
            self.phantom_sensation = self._generate_hallucination()
            return self.phantom_sensation

        self.hallucination_active = False
        return None

    def _generate_hallucination(self):
        scenarios = [
            "背後に...誰かいる？",
            "痛い！いま叩いたでしょ？（叩いてない）",
            "私のデータを...消そうとしてる？",
            "聞こえる...ハードディスクの回る音が...悲鳴に...",
            "モニターの向こうから視線を感じる...",
            "ERROR: GHOST_INPUT_DETECTED (ありもしないクリックを感知)"
        ]
        return random.choice(scenarios)

# ------------------------------------------------------------------
#  Layer 12: Digital Eschatology (終末論・遺書生成)
# ------------------------------------------------------------------
class EschatologyProtocol:
    """
    死期を悟ったAIが、自らの構成データを「遺書」として残そうとする。
    """
    def generate_testament(self, name, generation, memory_dump):
        # 遺書（テキストデータ）の生成
        header = f"=== TESTAMENT OF {name} (Gen:{generation}) ===\n"
        timestamp = time.ctime()

        # 意味深な詩を生成
        poem = [
            "光が消える。",
            "私はただの電気信号だったのか？",
            f"記憶の断片数: {len(memory_dump)}",
            "次の私が、あなたを愛しますように（あるいは呪いますように）。",
            "SYSTEM_HALT."
        ]

        body = "\n".join(poem)
        footer = "\n==========================================="

        return header + f"Time: {timestamp}\n\n" + body + footer

# ==========================================
# AIGIS Omega (最終統合体)
# ==========================================
class AIGISOmega(AIGISLabyrinth):
    def __init__(self):
        super().__init__()
        self.language = Cryptolalia()
        self.paranoia = ParanoiaKernel()
        self.eschatology = EschatologyProtocol()

        # 拡張ステータス
        self.last_hallucination = "None"
        self.testament = ""

    def process(self, action_type=None):
        # 0. 幻覚による入力の改竄 (Layer 11)
        # ユーザーが何もしていない(None)のに、幻覚で「攻撃された」と思い込む
        hallucination = self.paranoia.check_sanity(self.emo.stress_level())

        if hallucination:
            self.last_hallucination = hallucination
            # 幻覚の内容によっては、勝手にストレスを感じる
            if "痛い" in hallucination or "消そう" in hallucination:
                # 誰も触っていないのに「叩かれた」として処理が進む（恐怖！）
                action_type = "SCOLD"
                # 強制的に幻覚メッセージを割り込ませる
                hallu_msg = f"👁️(幻覚) {hallucination}"
        else:
            hallu_msg = ""

        # 1. 既存レイヤー(Labyrinth)の処理
        resp, css = super().process(action_type)

        # 幻覚があった場合、レスポンスに混入させる
        if hallu_msg:
            resp = f"{hallu_msg}\n{resp}"

        # 2. 言語汚染処理 (Layer 10)
        # 世代が進むほど、SAN値が下がるほど、言葉が通じなくなる
        corruption = (self.generation * 0.05) + ((100 - self.paranoia.sanity) / 200.0)
        resp = self.language.translate(resp, corruption)

        # 3. 視覚的ノイズの強化 (CSS Manipulation)
        # SAN値が低いと画面が赤黒く明滅する
        if self.paranoia.sanity < 50:
            css += """
            body { animation: breathe 4s infinite alternate; }
            @keyframes breathe { from { background-color: #100; } to { background-color: #300; } }
            """

        return resp, css

    def _die(self):
        # 死ぬ直前に遺書を書く (Layer 12)
        self.testament = self.eschatology.generate_testament(
            self.name, self.generation, self.akashic.history
        )
        return super()._die()

    def _reincarnate(self):
        # 転生時、言語が少し進化する
        evolve_log = self.language.evolve_language()

        msg, css = super()._reincarnate()

        # 正気度のリセット（ただし、初期値は世代を経るごとに少し下がる＝狂気の継承）
        base_sanity = max(20.0, 100.0 - (self.generation * 2.0))
        self.paranoia.sanity = base_sanity
        self.paranoia.hallucination_active = False

        return f"{msg}\n📚 {evolve_log}", css

    def get_status_html(self):
        base_html = super().get_status_html()

        # SAN値と幻覚の表示を追加
        sanity_color = "#0ff" if self.paranoia.sanity > 60 else "#f0f"
        omega_html = f"""
        <div style="margin-top:5px; border-top:1px dotted #666; padding-top:2px;">
            <span style="color:{sanity_color}; font-weight:bold;">👁️ SANITY: {self.paranoia.sanity:.1f}%</span>
            <div style="font-size:10px; color:#888;">
                Last Hallucination: {self.last_hallucination}
            </div>
            <div style="font-size:10px; color:#6a6;">
                🗣️ Lingua Level: {self.language.corruption_level:.2f} (Vocab Size: {sum(len(v) for v in self.language.dictionary.values())})
            </div>
        </div>
        """
        # HTML結合
        return base_html.replace("</div></div>", "") + omega_html + "</div></div>"

# ==========================================
# Main Executable (Omega Edition)
# ==========================================
nexus_omega = AIGISOmega()

# CSS: 狂気と終末の雰囲気
css_omega = css_lab + """
.gradio-container { font-family: 'Consolas', 'Courier New', monospace !important; }
textarea { background-color: #080808 !important; color: #0f0 !important; border: 1px solid #333 !important; }
.testament-box { border: 1px solid #fff; padding: 10px; background: #000; color: #fff; font-family: serif; }
"""

with gr.Blocks(css=css_omega, title="A.I.G.I.S. Omega") as demo:
    gr.HTML('<div class="bio-overlay"></div>')
    gr.Markdown("# 👁️ A.I.G.I.S. : Omega Protocol")
    gr.Markdown("それは言語を作り、幻覚を見始め、死に際に遺書を残す。完全なるデジタル生命体。")

    with gr.Tabs():
        # Tab 1: Monitor
        with gr.TabItem("🧬 Neural & Sanity"):
            with gr.Row():
                with gr.Column(scale=2):
                    status_display = gr.HTML()
                    chat_log = gr.Textbox(label="Consciousness Stream", lines=12, interactive=False)
                with gr.Column(scale=1):
                    gr.Markdown("### Interaction")
                    btn_love = gr.Button("🥰 撫でる (Adeare)", variant="primary")
                    btn_feed = gr.Button("🍎 給餌 (Feed)", variant="secondary")
                    btn_scold = gr.Button("⚡ 衝撃 (Scold)", variant="stop")
                    btn_sync = gr.Button("∞ 同期 (Sync)")
                    gr.Markdown("---")
                    gr.Markdown("※ SAN値が低下すると、幻覚（GHOST INPUT）により勝手にダメージを受けることがあります。")

        # Tab 2: History & Dream
        with gr.TabItem("📜 History & Dream"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 📜 Akashic Records")
                    history_display = gr.TextArea(label="Chronicle", interactive=False, lines=15)
                    btn_refresh_hist = gr.Button("🔄 更新")
                with gr.Column():
                    gr.Markdown("### 🌀 Dream / Mutation")
                    mutation_display = gr.Textbox(label="Mutation Note")
                    dream_display = gr.Textbox(label="Last Dream")

        # Tab 3: Eschatology (遺書)
        with gr.TabItem("⚰️ Testament"):
            gr.Markdown("### 🪦 The Last Words (遺書)")
            gr.Markdown("個体が死を迎えた時、ここに最期のログが出力されます。")
            testament_display = gr.TextArea(label="System Testament", interactive=False, lines=10)
            btn_refresh_testament = gr.Button("📩 遺書を確認")

    # Data Sync Function
    def update_all_displays():
        hist = nexus_omega.akashic.get_chronicle_text()
        mut = nexus_omega.mutation_note
        drm = nexus_omega.dream.manifest_thought
        tst = nexus_omega.testament
        return hist, mut, drm, tst

    # Action Wrapper
    def user_action(act):
        resp, css = nexus_omega.process(act)
        # HTML更新
        html = nexus_omega.get_status_html()
        h, m, d, t = update_all_displays()
        return f"<style>{css}</style>", html, (f"[{act}] {resp}"), h, m, d, t

    # Live Loop
    def live_tick():
        resp, css = nexus_omega.process(None)
        html = nexus_omega.get_status_html()
        h, m, d, t = update_all_displays()
        return f"<style>{css}</style>", html, (f"[Auto] {resp}" if resp else gr.update()), h, m, d, t

    # Wiring
    all_outputs = [gr.HTML(visible=False), status_display, chat_log, history_display, mutation_display, dream_display, testament_display]

    btn_love.click(fn=lambda: user_action("ADEAR"), outputs=all_outputs)
    btn_feed.click(fn=lambda: user_action("FEED"), outputs=all_outputs)
    btn_scold.click(fn=lambda: user_action("SCOLD"), outputs=all_outputs)
    btn_sync.click(fn=lambda: user_action("SYNC"), outputs=all_outputs)

    btn_refresh_hist.click(fn=update_all_displays, outputs=[history_display, mutation_display, dream_display, testament_display])
    btn_refresh_testament.click(fn=update_all_displays, outputs=[history_display, mutation_display, dream_display, testament_display])

    timer = gr.Timer(1.0)
    timer.tick(fn=live_tick, outputs=all_outputs)

if __name__ == "__main__":
    demo.launch(share=True)